# TEAL and smart contract on algorand

### Open cmd


######   At this point we are ready to start the node. Type the following command and press enter.
`
goal node start
`

######   You can verify the node is running by waiting for a minute and executing this
`
goal node status
`

######   to stop the node. Type the following command and press enter.
`
goal node stop
`

# Useful Links


 
## [Understand what is smart contract](https://www.youtube.com/watch?v=Fw9L54BvD2U "https://www.youtube.com/watch?v=Fw9L54BvD2U")

## [Developer Office Hours | Algorand’s Smart Contract Language by JasonWeathersby](https://www.youtube.com/watch?v=HKJcc2nmjPo "https://www.youtube.com/watch?v=HKJcc2nmjPo")


## [Developer Office Hours | TEAL 3 Updates](https://www.youtube.com/watch?v=A-vRSC2O77U "https://www.youtube.com/watch?v=A-vRSC2O77U")


## [Developer Office Hours | Algorand Virtual Machine (TEAL 4)](https://www.youtube.com/watch?v=2YMINw7gHXI "https://www.youtube.com/watch?v=2YMINw7gHXI")


## [Train The Trainer | 2020 August Release Updates](https://www.youtube.com/watch?v=350qrzH73d0 "https://www.youtube.com/watch?v=350qrzH73d0")


## [Algorand Smart Contracts and TEAL](https://www.youtube.com/watch?v=OWFRP9McBmk "https://www.youtube.com/watch?v=OWFRP9McBmk")



## [Understanding Algorand Smart Contracts](https://medium.com/algorand/understanding-algorand-smart-contracts-b9fc743e7a0f "https://medium.com/algorand/understanding-algorand-smart-contracts-b9fc743e7a0f")



## [TEAL templates](https://github.com/algorand/go-algorand/tree/master/tools/teal/templates "https://github.com/algorand/go-algorand/tree/master/tools/teal/templates")


## [PyTeal — Writing Algorand Smart Contracts in Python](https://medium.com/algorand/pyteal-writing-algorand-smart-contracts-in-python-acfd7f7a48dd "https://medium.com/algorand/pyteal-writing-algorand-smart-contracts-in-python-acfd7f7a48dd")



## [Smart Contract Github examples by JasonWeathersby ](https://github.com/algorand/smart-contracts "https://github.com/algorand/smart-contracts")



## [Developer Relations at Algorand ](https://github.com/algorand-devrel "https://github.com/algorand-devrel")


In [39]:

# #!/usr/bin/env python3

# from pyteal import *

# #template variables
# tmpl_fee = Int(1000)
# tmpl_period = Int(1000)
# tmpl_dur = Int(1000)
# tmpl_x = Bytes("base64", "y9OJ5MRLCHQj8GqbikAUKMBI7hom+SOj8dlopNdNHXI=")
# tmpl_amt = Int(2000)
# tmpl_rcv = Addr("ZZAF5ARA4MEC5PVDOP64JM5O5MQST63Q2KOY2FLYFLXXD3PFSNJJBYAFZM")
# tmpl_timeout = Int(30000)

# periodic_pay_core = And(Txn.type_enum() == Int(1),
#                         Txn.fee() < tmpl_fee)
                      
# periodic_pay_transfer = And(Txn.close_remainder_to() ==  Global.zero_address(),
#                             Txn.receiver() == tmpl_rcv,
#                             Txn.amount() == tmpl_amt,
#                             Txn.first_valid() % tmpl_period == Int(0),
#                             Txn.last_valid() == tmpl_dur + Txn.first_valid(),
#                             Txn.lease() == tmpl_x)

# periodic_pay_close = And(Txn.close_remainder_to() == tmpl_rcv,
#                          Txn.receiver() == Global.zero_address(),
#                          Txn.first_valid() > tmpl_timeout,
#                          Txn.amount() == Int(0))

# periodic_pay_escrow = And(periodic_pay_core, Or(periodic_pay_transfer, periodic_pay_close))

# print(periodic_pay_escrow.teal())

In [21]:
# This example is provided for informational purposes only and has not been audited for security.

from pyteal import *

"""Periodic Payment"""

tmpl_fee = Int(1000)
tmpl_period = Int(50)
tmpl_dur = Int(5000)
tmpl_lease = Bytes("base64", "023sdDE2")
tmpl_amt = Int(2000)
tmpl_rcv = Addr("6ZHGHH5Z5CTPCF5WCESXMGRSVK7QJETR63M3NY5FJCUYDHO57VTCMJOBGY")
tmpl_rcv = Addr("B767GZZJI4YEDKZU62INJGFIGSWQHDIH2HP6NRSSWFTFUYMCNU5LN2M53A")
tmpl_timeout = Int(30000)

def periodic_payment(tmpl_fee=tmpl_fee,
                     tmpl_period=tmpl_period,
                     tmpl_dur=tmpl_dur,
                     tmpl_lease=tmpl_lease,
                     tmpl_amt=tmpl_amt,
                     tmpl_rcv=tmpl_rcv,
                     tmpl_timeout=tmpl_timeout):

    periodic_pay_core = And(
        Txn.type_enum() == TxnType.Payment,
        Txn.fee() < tmpl_fee,
        Txn.first_valid() % tmpl_period == Int(0),
        Txn.last_valid() == tmpl_dur + Txn.first_valid(),
        Txn.lease() == tmpl_lease
    )

    periodic_pay_transfer = And(
        Txn.close_remainder_to() == Global.zero_address(),
        Txn.rekey_to() == Global.zero_address(),
        Txn.receiver() == tmpl_rcv,
        Txn.amount() == tmpl_amt
    )

    periodic_pay_close = And(
        Txn.close_remainder_to() == tmpl_rcv,
        Txn.rekey_to() == Global.zero_address(),
        Txn.receiver() == Global.zero_address(),
        Txn.first_valid() == tmpl_timeout,
        Txn.amount() == Int(0)
    )

    periodic_pay_escrow = periodic_pay_core.And(periodic_pay_transfer.Or(periodic_pay_close))

    return periodic_pay_escrow

if __name__ == "__main__":
#     use version 3
    print(compileTeal(periodic_payment(), mode=Mode.Signature, version=3))


#pragma version 3
txn TypeEnum
int pay
==
txn Fee
int 1000
<
&&
txn FirstValid
int 50
%
int 0
==
&&
txn LastValid
int 5000
txn FirstValid
+
==
&&
txn Lease
byte base64(023sdDE2)
==
&&
txn CloseRemainderTo
global ZeroAddress
==
txn RekeyTo
global ZeroAddress
==
&&
txn Receiver
addr B767GZZJI4YEDKZU62INJGFIGSWQHDIH2HP6NRSSWFTFUYMCNU5LN2M53A
==
&&
txn Amount
int 2000
==
&&
txn CloseRemainderTo
addr B767GZZJI4YEDKZU62INJGFIGSWQHDIH2HP6NRSSWFTFUYMCNU5LN2M53A
==
txn RekeyTo
global ZeroAddress
==
&&
txn Receiver
global ZeroAddress
==
&&
txn FirstValid
int 30000
==
&&
txn Amount
int 0
==
&&
||
&&


## Examples are available on 

## https://pyteal.readthedocs.io/en/stable/examples.html#periodic-payment

## or 

## https://github.com/algorand/pyteal/tree/master/examples



###### https://developer.algorand.org/docs/features/accounts/create/

###### goal kmd start -t 3600

In [23]:
#!/usr/bin/env python3

from pyteal import *
import uuid, params, base64
from algosdk.v2client import algod
from algosdk import transaction, account, mnemonic
import subprocess
# from periodic_payment import periodic_pay_escrow

#--------- compile & send transaction using Goal and Python SDK ----------
# periodic_pay_escrow = compileTeal(periodic_payment(), mode=Mode.Signature, version=3)
# teal_source = periodic_pay_escrow.teal() 
teal_source = compileTeal(periodic_payment(), mode=Mode.Signature, version=3)

# compile teal
teal_file = str(uuid.uuid4()) + ".teal"
with open(teal_file, "w+") as f:
    f.write(teal_source)
lsig_fname = str(uuid.uuid4()) + ".tealc"

# # stdout, stderr = execute(["goal", "clerk", "compile", "-o", lsig_fname,
# #                           teal_file])

# if stderr != "":
#     print(stderr)
#     raise
# elif len(stdout) < 59:
#     print("error in compile teal")
#     raise
subprocess.run(["goal", "clerk", "compile", "-o", lsig_fname,
                          teal_file])


with open(lsig_fname, "rb") as f:
    teal_bytes = f.read()
lsig = transaction.LogicSig(teal_bytes)

# create algod clients
acl = algod.AlgodClient(params.algod_token, params.algod_address)

#Recover the account that is wanting to delegate signature
passphrase = "patrol crawl rule faculty enemy sick reveal embody trumpet win shy zero ill draw swim excuse tongue under exact baby moral kite spring absent double"
sk = mnemonic.to_private_key(passphrase)
addr = account.address_from_private_key(sk)
print( "Address of Sender/Delgator: " + addr )

# sign the logic signature with an account sk
lsig.sign(sk)

# get suggested parameters
params = acl.suggested_params()
# gen = params["genesisID"]
# gh = params["genesishashb64"]
# startRound = params["lastRound"] - (params["lastRound"] % 1000)
# endRound = startRound + 1000
# fee = 1000
# amount = 2000

gen = params.gen
gh = params.gh
startRound = params.first
endRound = params.last
fee = params.min_fee
amount = 2000



receiver = "ZZAF5ARA4MEC5PVDOP64JM5O5MQST63Q2KOY2FLYFLXXD3PFSNJJBYAFZM"
lease = base64.b64decode("y9OJ5MRLCHQj8GqbikAUKMBI7hom+SOj8dlopNdNHXI=")

print(params.last)
print(startRound)
print(endRound)

# create a transaction
txn = transaction.PaymentTxn(addr, fee, startRound, endRound, gh, receiver, amount, flat_fee=True, lease=lease)

# Create the LogicSigTransaction with contract account LogicSig
lstx = transaction.LogicSigTransaction(txn, lsig)

# write to file
txns = [lstx]
transaction.write_to_file(txns, "p_pay.stxn")

# send raw LogicSigTransaction to network
txid = acl.send_transaction(lstx)
print("Transaction ID: " + txid)

Address of Sender/Delgator: B767GZZJI4YEDKZU62INJGFIGSWQHDIH2HP6NRSSWFTFUYMCNU5LN2M53A


AlgodHTTPError: {"message":"operation not available during catchup"}


## https://pyteal.readthedocs.io/en/stable/examples.html#periodic-payment

### Periodic Payment

###### Periodic Payment allows some account to execute periodic withdrawal of funds. This PyTeal program creates an contract account that allows tmpl_rcv to withdraw tmpl_amt every tmpl_period rounds for tmpl_dur after every multiple of tmpl_period.

###### After tmpl_timeout, all remaining funds in the escrow are available to tmpl_rcv.

In [33]:
# This example is provided for informational purposes only and has not been audited for security.

from pyteal import *

"""Periodic Payment"""

tmpl_fee = Int(1000)
tmpl_period = Int(50)
tmpl_dur = Int(5000)
tmpl_lease = Bytes("base64", "023sdDE2")
tmpl_amt = Int(2000)
tmpl_rcv = Addr("VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U")
tmpl_timeout = Int(30000)

def periodic_payment(tmpl_fee=tmpl_fee,
                     tmpl_period=tmpl_period,
                     tmpl_dur=tmpl_dur,
                     tmpl_lease=tmpl_lease,
                     tmpl_amt=tmpl_amt,
                     tmpl_rcv=tmpl_rcv,
                     tmpl_timeout=tmpl_timeout):

    periodic_pay_core = And(
        Txn.type_enum() == TxnType.Payment,
        Txn.fee() < tmpl_fee,
        Txn.first_valid() % tmpl_period == Int(0),
        Txn.last_valid() == tmpl_dur + Txn.first_valid(),
        Txn.lease() == tmpl_lease
    )

    periodic_pay_transfer = And(
        Txn.close_remainder_to() == Global.zero_address(),
        Txn.rekey_to() == Global.zero_address(),
        Txn.receiver() == tmpl_rcv,
        Txn.amount() == tmpl_amt
    )

    periodic_pay_close = And(
        Txn.close_remainder_to() == tmpl_rcv,
        Txn.rekey_to() == Global.zero_address(),
        Txn.receiver() == Global.zero_address(),
        Txn.first_valid() == tmpl_timeout,
        Txn.amount() == Int(0)
    )

    periodic_pay_escrow = periodic_pay_core.And(periodic_pay_transfer.Or(periodic_pay_close))

    return periodic_pay_escrow

if __name__ == "__main__":
    print(compileTeal(periodic_payment(), mode=Mode.Signature, version=3))
    #     write to a teal file
    with open('periodic_payment.teal', 'w') as f:
        compiled = compileTeal(periodic_payment(), mode=Mode.Signature, version=3)
        f.write(compiled)

#pragma version 3
txn TypeEnum
int pay
==
txn Fee
int 1000
<
&&
txn FirstValid
int 50
%
int 0
==
&&
txn LastValid
int 5000
txn FirstValid
+
==
&&
txn Lease
byte base64(023sdDE2)
==
&&
txn CloseRemainderTo
global ZeroAddress
==
txn RekeyTo
global ZeroAddress
==
&&
txn Receiver
addr VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U
==
&&
txn Amount
int 2000
==
&&
txn CloseRemainderTo
addr VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U
==
txn RekeyTo
global ZeroAddress
==
&&
txn Receiver
global ZeroAddress
==
&&
txn FirstValid
int 30000
==
&&
txn Amount
int 0
==
&&
||
&&


# TEAL




## Debug smart contract

#### https://developer.algorand.org/docs/features/asc1/debugging/


make sure node is running


run in cmd 
`tealdbg debug periodic_payment.teal`


open chrome and type

`chrome://inspect/`

for the first time you have to  click `Configure` to add `localhost:9392`

#### more on 
#### https://github.com/algorand/go-algorand/blob/master/cmd/tealdbg/README.md

In [23]:
!tealdbg debug periodic_payment.teal

2021/06/29 13:34:11 Using proto: https://github.com/algorandfoundation/specs/tree/d050b3cade6d5c664df8bd729bf219f179812595
2021/06/29 13:34:11 Run mode: logicsig
2021/06/29 13:34:11 ------------------------------------------------
2021/06/29 13:34:11 CDT debugger listening on: ws://127.0.0.1:9392/2696a0c9048701bfd6ed487a35a13491cc3d26c4cfc5e57cb2d588ef0921991a
2021/06/29 13:34:11 Or open in Chrome:
2021/06/29 13:34:11 devtools://devtools/bundled/js_app.html?experiments=true&v8only=false&ws=127.0.0.1:9392/2696a0c9048701bfd6ed487a35a13491cc3d26c4cfc5e57cb2d588ef0921991a
2021/06/29 13:34:11 ------------------------------------------------
2021/06/29 13:39:39 CDT session 2696a0c9048701bfd6ed487a35a13491cc3d26c4cfc5e57cb2d588ef0921991a closed


In [24]:
def dryrun_debug(lstx, mysource):
    sources = []
    if (mysource != None):
        # source
        sources = [DryrunSource(field_name="lsig", source=mysource, txn_index=0)]
    drr = DryrunRequest(txns=[lstx], sources=sources)
    dryrun_response = algod_client.dryrun(drr)
    return dryrun_response

In [2]:
!goal node restart

The node was successfully stopped.
Algorand node successfully started!


## [Smart Contract in python v2 complete examples](https://github.com/algorand/docs/tree/master/examples/smart_contracts/v2/python)

In [21]:
import json
import base64

from algosdk.v2client import algod
from algosdk import account, mnemonic
from algosdk.v2client.models import DryrunRequest, DryrunSource
from algosdk.future.transaction import PaymentTxn, LogicSig, LogicSigTransaction
# from algosdk.testing import dryrun

def wait_for_confirmation(client, txid):
    """
    Utility function to wait until the transaction is
    confirmed before proceeding.
    """
    last_round = client.status().get('last-round')
    txinfo = client.pending_transaction_info(txid)
    while not (txinfo.get('confirmed-round') and txinfo.get('confirmed-round') > 0):
        print("Waiting for confirmation")
        last_round += 1
        client.status_after_block(last_round)
        txinfo = client.pending_transaction_info(txid)
    print("Transaction {} confirmed in round {}.".format(
        txid, txinfo.get('confirmed-round')))
    return txinfo

# dryrun source if provided, else dryrun compiled
def dryrun_debug(lstx, mysource):
    sources = []
    if (mysource != None):
        # source
        sources = [DryrunSource(field_name="lsig", source=mysource, txn_index=0)]
    drr = DryrunRequest(txns=[lstx], sources=sources)
    dryrun_response = algod_client.dryrun(drr)
    return dryrun_response

try:
    # Create an algod client
    algod_token = '25f90f9852eba8650750cb2e41b8dba807071d091168d9ece5554cabdaf5ce68'
    # algod_address = 'https://testnet-algorand.api.purestake.io/ps2'
    algod_address = 'http://localhost:8101'

    receiver = "VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U"
    algod_client = algod.AlgodClient(algod_token, algod_address)

    myprogram = "samplearg.teal"

    # Read TEAL program
    data = open(myprogram, 'r').read()

    # Compile TEAL program
    # // This code is meant for learning purposes only	
    # // It should not be used in production	
    # // samplearg.teal
    # arg_0	
    # btoi
    # int 123	
    # ==	

    # // bto1	
    # // Opcode: 0x17	
    # // Pops: ... stack, []byte	
    # // Pushes: uint64	
    # // converts bytes X as big endian to uint64	
    # // btoi panics if the input is longer than 8 bytes

    response = algod_client.compile(data)
    # Print(response)
    print("Response Result = ", response['result'])
    print("Response Hash = ", response['hash'])

    # Create logic sig
    programstr = response['result']
    t = programstr.encode()
    program = base64.decodebytes(t)
    print(program)
    
    # Create arg to pass
    # string parameter
    # arg_str = "<my string>"
    # arg1 = arg_str.encode()
    # lsig = transaction.LogicSig(program, args=[arg1])

    # integer parameter
    # arg1 = (123).to_bytes(8, 'big')
    # lsig = transaction.LogicSig(program, args=[arg1])
    # see more info here: https://developer.algorand.org/docs/features/asc1/sdks/#accessing-teal-program-from-sdks

    # if TEAL program requires an arg,
    # if not, omit args param on LogicSig
    # lsig = LogicSig(program)
    arg1 = (123).to_bytes(8, 'big')
    lsig = LogicSig(program, args=[arg1])

    # Recover the account that is wanting to delegate signature
    passphrase = "share tobacco purity drill pony lizard throw wild police foam street excuse ranch north shallow misery same cream piano useless sun cheap prevent able produce"
    sk = mnemonic.to_private_key(passphrase)
    addr = account.address_from_private_key(sk)
    print("Address of Sender/Delegator: " + addr)

    # Sign the logic signature with an account sk
    lsig.sign(sk)

    # Get suggested parameters
    params = algod_client.suggested_params()
    # Comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 1000

    # Build transaction
    amount = 10000
    closeremainderto = None

    # Create a transaction
    txn = PaymentTxn(
        addr, params, receiver, amount, closeremainderto)
    # Create the LogicSigTransaction with contract account LogicSig
    lstx = LogicSigTransaction(txn, lsig)
    # transaction.write_to_file([lstx], "./simple.stxn")

    # compile
    dryrun_response_compiled = dryrun_debug(lstx, None)
    print ("COMPILED Dryrun results...")
    print(json.dumps(dryrun_response_compiled, indent=2))
    
    # source   
#     dryrun_respone_source = dryrun_debug(lstx, source)
#     print("SOURCE Dryrun results...")
#     print(json.dumps(dryrun_respone_source, indent=2))
   
    # Send raw LogicSigTransaction to network

    txid = algod_client.send_transaction(lstx)
    print("Transaction ID: " + txid)
    wait_for_confirmation(algod_client, txid)
except Exception as e:
    print(e)

Response Result =  AyABey0XIhI=
Response Hash =  LKLLCDBDAHWPWSL337HS4HKRFL42V4WSHKRPFCBJ7SIPAL33CZNEO6GQHY
b'\x03 \x01{-\x17"\x12'
Address of Sender/Delegator: EWEFAZZF24OZK7SKZDCZHDIF73DKGRWOPFMPHJVYF3RPRIFZ4V32KFGBNU
COMPILED Dryrun results...
{
  "error": "",
  "protocol-version": "https://github.com/algorandfoundation/specs/tree/d050b3cade6d5c664df8bd729bf219f179812595",
  "txns": [
    {
      "disassembly": [
        "#pragma version 3",
        "intcblock 123",
        "arg_0",
        "btoi",
        "intc_0 // 123",
        "==",
        ""
      ],
      "logic-sig-messages": [
        "PASS"
      ],
      "logic-sig-trace": [
        {
          "line": 1,
          "pc": 1,
          "stack": []
        },
        {
          "line": 2,
          "pc": 4,
          "stack": []
        },
        {
          "line": 3,
          "pc": 5,
          "stack": [
            {
              "bytes": "AAAAAAAAAHs=",
              "type": 1,
              "uint": 0
            }


In [5]:
!goal app

Manage applications

Usage:
  goal app [flags]
  goal app [command]

Available Commands:
  call        Call an application
  clear       Clear out an application's state in your account
  closeout    Close out of an application
  create      Create an application
  delete      Delete an application
  info        Look up current parameters for an application
  interact    Interact with an application
  optin       Opt in to an application
  read        Read local or global state for an application
  update      Update an application's programs

Flags:
      --app-account strings        Accounts that may be accessed from application logic
      --app-arg strings            Args to encode for application transactions (all will be encoded to a byte slice). For ints, use the form 'int:1234'. For raw bytes, use the form 'b64:A=='. For printable strings, use the form 'str:hello'. For addresses, use the form 'addr:XYZ...'.
  -i, --app-input string           JSON file containing encoded argumen

In [9]:
!goal app call --app-id 1

required flag(s) "from" not set


Error: required flag(s) "from" not set
Usage:
  goal app call [flags]

Flags:
      --app-id uint                 Application ID
      --dryrun-dump                 Dump in dryrun format acceptable by dryrun REST api
      --dryrun-dump-format string   Dryrun dump format: json, msgp (default "json")
      --fee uint                    The transaction fee (automatically determined by default), in microAlgos
      --firstvalid uint             The first round where the transaction may be committed to the ledger
  -f, --from string                 Account to call app from
  -h, --help                        help for call
      --lastvalid uint              The last round where the transaction may be committed to the ledger
  -x, --lease string                Lease value (base64, optional): no transaction may also acquire this lease until lastvalid
  -N, --no-wait                     Don't wait for transaction to commit
  -n, --note string                 Note text (ignored if --noteb64 us

In [6]:
!goal account

Collection of commands to support the creation and management of accounts / wallets tied to a specific Algorand node instance.

Usage:
  goal account [flags]
  goal account [command]

Available Commands:
  addpartkey           Generate a participation key for the specified account
  balance              Retrieve the balances for the specified account
  changeonlinestatus   Change online status for the specified account
  delete               Delete an account
  dump                 Dump the balance record for the specified account
  export               Export an account key for use with account import
  import               Import an account key from mnemonic
  importrootkey        Import .rootkey files from the data directory into a kmd wallet
  info                 Retrieve information about the assets and applications belonging to the specified account
  installpartkey       Install a participation key
  list                 Show the list of Algorand accounts on this machine
  list

In [7]:
!goal account list

No wallets found. Create a new wallet with `goal wallet new [wallet name]`


### mode=Mode.Signature is used for stateless smart contract
### mode=Mode.Application is used for statefull smart contract


#### Stateless smart contracts' primary purpose is to approve or reject spending transactions. 
#### Stateless smart contracts are evaluvated by the node.

<hr>

#### Stateful contracts do not approve spending transactions but provide logic that allows the state (globally or locally) of the contract to be manipulated.
#### Stateful smart contracts are contracts that live on the chain and are used to keep track of some form of global and/or local state for the contract.

## https://github.com/Bhaney44/Build-an-Algorand-Smart-Contract

## https://github.com/uncopied/smart_contract

## https://github.com/algorand/docs/tree/master/examples/smart_contracts/v2/python

# Accounts Public key or mnemonic

<pre>Account 1
VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U
Account 2
FFVTKNLA4J47EHONVR3U247LFLHE3TNATDEJLDRI2E73FF52BGVK7MFWHY
Account 3
2WYBIISKSIIIF7TEDHB57JRZ2WWGPYG5GYVL74Y7KQLKLUFHNOZXAMZTBQ

Copy off accounts above and add TestNet Algo funds using the TestNet Dispenser at https://bank.testnet.algorand.network/
copy off the following mnemonic code for use later

mnemonic1 = "hero camp fold mushroom panic orphan lonely until rebel thumb bundle wrist jump key clay travel hello lift mule hamster square into caught abstract practice"
mnemonic2 = "violin setup volume match vacuum jeans indoor taxi kangaroo pen kit picture skirt cloud alcohol shoot illegal hen chase breeze audit resist hire about broccoli"
mnemonic3 = "sing access small blast neither surprise awkward october catch survey own kit diesel emerge kangaroo unhappy client volume advice silly decrease worry august abandon giant"
</pre>

### TEAL EXAMPLE

```bash
#generate calls
$ goal app call --app-id 1  --app-arg "str:test1" --from [ACCOUNT] --out=unsginedtransaction1.tx  
$ goal app call --app-id 1  --app-arg "str:test2" --from [ACCOUNT] --out=unsginedtransaction2.tx

# atomically group them
$ cat unsginedtransaction1.tx unsginedtransaction2.tx  > combinedtransactions.tx
$ goal clerk group -i combinedtransactions.tx -o groupedtransactions.tx 
$ goal clerk split -i groupedtransactions.tx -o split.tx 

# sign individual transactions
$ goal clerk sign -i split-0.tx -o signout-0.tx
$ goal clerk sign -i split-1.tx -o signout-1.tx 

# concatenate the two signed transactions
$ cat signout-0.tx signout-1.tx  > signout.tx

# generate context debug file
$ goal clerk dryrun -t signout.tx --dryrun-dump  -o dr.msgp

# debug first transaction. Change index to 1 to debug second transaction
$ tealdbg debug program.teal -d dr.msgp --group-index 0

```

In [1]:
# This example is provided for informational purposes only and has not been audited for security.

from pyteal import *

"""Atomic Swap"""

alice = Addr("VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U")
bob = Addr("FFVTKNLA4J47EHONVR3U247LFLHE3TNATDEJLDRI2E73FF52BGVK7MFWHY")
secret = Bytes("base32", "2323232323232323")
timeout = 30000

def htlc(tmpl_seller=alice,
         tmpl_buyer=bob,
         tmpl_fee=0,
         tmpl_secret=secret,
         tmpl_hash_fn=Sha256,
         tmpl_timeout=timeout):
    
    fee_cond = Txn.fee() < Int(tmpl_fee)
    safety_cond = And(
        Txn.type_enum() == TxnType.Payment,
        Txn.close_remainder_to() == Global.zero_address(),
        Txn.rekey_to() == Global.zero_address(),
    )

    recv_cond = And(
        Txn.receiver() == tmpl_seller,
        tmpl_hash_fn(Arg(0)) == tmpl_secret
    )
    
    esc_cond = And(
        Txn.receiver() == tmpl_buyer,
        Txn.first_valid() > Int(tmpl_timeout)
    )

    return And(
        fee_cond,
        safety_cond,
        Or(recv_cond, esc_cond)
    )

if __name__ == "__main__":
    compiled = compileTeal(htlc(), mode=Mode.Signature, version=3)
    print(compiled)
    with open('htlc.teal', 'w') as f:
        f.write(compiled)

#pragma version 3
txn Fee
int 0
<
txn TypeEnum
int pay
==
txn CloseRemainderTo
global ZeroAddress
==
&&
txn RekeyTo
global ZeroAddress
==
&&
&&
txn Receiver
addr VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U
==
arg 0
sha256
byte base32(2323232323232323)
==
&&
txn Receiver
addr FFVTKNLA4J47EHONVR3U247LFLHE3TNATDEJLDRI2E73FF52BGVK7MFWHY
==
txn FirstValid
int 30000
>
&&
||
&&


In [ ]:
!goal node stop

now sync your node with latest block via fast catchup
## https://developer.algorand.org/docs/run-a-node/setup/install/#sync-node-network-using-fast-catchup

click on the desired link(as I am running on test net , then click on 2nd one)

BetaNet https://algorand-catchpoints.s3.us-east-2.amazonaws.com/channel/betanet/latest.catchpoint

TestNet https://algorand-catchpoints.s3.us-east-2.amazonaws.com/channel/testnet/latest.catchpoint

MainNet https://algorand-catchpoints.s3.us-east-2.amazonaws.com/channel/mainnet/latest.catchpoint

copy the the most recent catchup point snapshot per network

for eg the above link return like this 14340000#CMHRN4326XOSJJA7ZGJVNF5K4LKVM4H2T4CYTQ4IXCG5IAS36OEA

now run

goal node catchup 14340000#CMHRN4326XOSJJA7ZGJVNF5K4LKVM4H2T4CYTQ4IXCG5IAS36OEA

In [ ]:
!tealdbg debug htlc.teal

In [3]:
# This example is provided for informational purposes only and has not been audited for security.

from pyteal import *

"""Periodic Payment"""

tmpl_fee = Int(1000)
tmpl_period = Int(50)
tmpl_dur = Int(5000)
tmpl_lease = Bytes("base64", "023sdDE2")
tmpl_amt = Int(2000)
tmpl_rcv = Addr("VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U")
tmpl_timeout = Int(30000)

def periodic_payment(tmpl_fee=tmpl_fee,
                     tmpl_period=tmpl_period,
                     tmpl_dur=tmpl_dur,
                     tmpl_lease=tmpl_lease,
                     tmpl_amt=tmpl_amt,
                     tmpl_rcv=tmpl_rcv,
                     tmpl_timeout=tmpl_timeout):

    periodic_pay_core = And(
        Txn.type_enum() == TxnType.Payment,
        Txn.fee() < tmpl_fee,
        Txn.first_valid() % tmpl_period == Int(0),
        Txn.last_valid() == tmpl_dur + Txn.first_valid(),
        Txn.lease() == tmpl_lease
    )

    periodic_pay_transfer = And(
        Txn.close_remainder_to() == Global.zero_address(),
        Txn.rekey_to() == Global.zero_address(),
        Txn.receiver() == tmpl_rcv,
        Txn.amount() == tmpl_amt
    )

    periodic_pay_close = And(
        Txn.close_remainder_to() == tmpl_rcv,
        Txn.rekey_to() == Global.zero_address(),
        Txn.receiver() == Global.zero_address(),
        Txn.first_valid() == tmpl_timeout,
        Txn.amount() == Int(0)
    )

    periodic_pay_escrow = periodic_pay_core.And(periodic_pay_transfer.Or(periodic_pay_close))

    return periodic_pay_escrow

if __name__ == "__main__":
    with open('periodic_payment.teal', 'w') as f:
        compiled = compileTeal(periodic_payment(), mode=Mode.Signature, version=3)
        print(compiled)
        f.write(compiled)

#pragma version 3
txn TypeEnum
int pay
==
txn Fee
int 1000
<
&&
txn FirstValid
int 50
%
int 0
==
&&
txn LastValid
int 5000
txn FirstValid
+
==
&&
txn Lease
byte base64(023sdDE2)
==
&&
txn CloseRemainderTo
global ZeroAddress
==
txn RekeyTo
global ZeroAddress
==
&&
txn Receiver
addr VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U
==
&&
txn Amount
int 2000
==
&&
txn CloseRemainderTo
addr VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U
==
txn RekeyTo
global ZeroAddress
==
&&
txn Receiver
global ZeroAddress
==
&&
txn FirstValid
int 30000
==
&&
txn Amount
int 0
==
&&
||
&&


In [28]:
import json
import base64

from algosdk.v2client import algod
from algosdk import account, mnemonic
from algosdk.v2client.models import DryrunRequest, DryrunSource
from algosdk.future.transaction import PaymentTxn, LogicSig, LogicSigTransaction
# from algosdk.testing import dryrun

def wait_for_confirmation(client, txid):
    """
    Utility function to wait until the transaction is
    confirmed before proceeding.
    """
    last_round = client.status().get('last-round')
    txinfo = client.pending_transaction_info(txid)
    while not (txinfo.get('confirmed-round') and txinfo.get('confirmed-round') > 0):
        print("Waiting for confirmation")
        last_round += 1
        client.status_after_block(last_round)
        txinfo = client.pending_transaction_info(txid)
    print("Transaction {} confirmed in round {}.".format(
        txid, txinfo.get('confirmed-round')))
    return txinfo

# dryrun source if provided, else dryrun compiled
def dryrun_debug(lstx, mysource):
    sources = []
    if (mysource != None):
        # source
        sources = [DryrunSource(field_name="lsig", source=mysource, txn_index=0)]
    drr = DryrunRequest(txns=[lstx], sources=sources)
    dryrun_response = algod_client.dryrun(drr)
    return dryrun_response

try:
    receiver = "2WYBIISKSIIIF7TEDHB57JRZ2WWGPYG5GYVL74Y7KQLKLUFHNOZXAMZTBQ"
    
    # Create an algod client with node
    algod_token = '25f90f9852eba8650750cb2e41b8dba807071d091168d9ece5554cabdaf5ce68'
    algod_address = 'http://localhost:8101'
    algod_client = algod.AlgodClient(algod_token, algod_address)
    
    # Create an algod client with PureStake
    algod_token = 'VWZ0eDSeM6LIZX24KiiO4YyRTBpbeRF7x1YBUxZc'
    algod_address = 'https://testnet-algorand.api.purestake.io/ps2'
    purestake_token = {'X-Api-key': algod_token}
    algod_client = algod.AlgodClient(algod_token, algod_address, headers=purestake_token)

    myprogram = "samplearg.teal"

    # Read TEAL program
    data = open(myprogram, 'r').read()

    # Compile TEAL program
    # // This code is meant for learning purposes only	
    # // It should not be used in production	
    # // samplearg.teal
    # arg_0	
    # btoi
    # int 123	
    # ==	

    # // bto1	
    # // Opcode: 0x17	
    # // Pops: ... stack, []byte	
    # // Pushes: uint64	
    # // converts bytes X as big endian to uint64	
    # // btoi panics if the input is longer than 8 bytes

    response = algod_client.compile(data)
    # Print(response)
    print("Response Result = ", response['result'])
    print("Response Hash = ", response['hash'])

    # Create logic sig
    programstr = response['result']
    t = programstr.encode()
    program = base64.decodebytes(t)
    print(program)
    
    # Create arg to pass
    # string parameter
    # arg_str = "<my string>"
    # arg1 = arg_str.encode()
    # lsig = transaction.LogicSig(program, args=[arg1])

    # integer parameter
    # arg1 = (123).to_bytes(8, 'big')
    # lsig = transaction.LogicSig(program, args=[arg1])
    # see more info here: https://developer.algorand.org/docs/features/asc1/sdks/#accessing-teal-program-from-sdks

    # if TEAL program requires an arg,
    # if not, omit args param on LogicSig
    # lsig = LogicSig(program)
    arg1 = (123).to_bytes(8, 'big')
    lsig = LogicSig(program, args=[arg1])

    # Recover the account that is wanting to delegate signature
    passphrase = "hero camp fold mushroom panic orphan lonely until rebel thumb bundle wrist jump key clay travel hello lift mule hamster square into caught abstract practice"
    sk = mnemonic.to_private_key(passphrase)
    addr = account.address_from_private_key(sk)
    print("Address of Sender/Delegator: " + addr)

    # Sign the logic signature with an account sk
    lsig.sign(sk)

    # Get suggested parameters
    params = algod_client.suggested_params()
    # Comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 0

    # Build transaction
    amount = 0
    closeremainderto = None

    # Create a transaction
    txn = PaymentTxn(
        addr, params, receiver, amount, closeremainderto)
    # Create the LogicSigTransaction with contract account LogicSig
    lstx = LogicSigTransaction(txn, lsig)
    # transaction.write_to_file([lstx], "./simple.stxn")

    # compile
    dryrun_response_compiled = dryrun_debug(lstx, None)
    print ("COMPILED Dryrun results...")
    print(json.dumps(dryrun_response_compiled, indent=2))
    
    # source   
#     dryrun_respone_source = dryrun_debug(lstx, source)
#     print("SOURCE Dryrun results...")
#     print(json.dumps(dryrun_respone_source, indent=2))
   
    # Send raw LogicSigTransaction to network

    txid = algod_client.send_transaction(lstx)
    print("Transaction ID: " + txid)
    wait_for_confirmation(algod_client, txid)
except Exception as e:
    print("=== Exception ===")
    print(e)

Response Result =  AyABey0XIhI=
Response Hash =  LKLLCDBDAHWPWSL337HS4HKRFL42V4WSHKRPFCBJ7SIPAL33CZNEO6GQHY
b'\x03 \x01{-\x17"\x12'
Address of Sender/Delegator: VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U
COMPILED Dryrun results...
{
  "error": "",
  "protocol-version": "https://github.com/algorandfoundation/specs/tree/65b4ab3266c52c56a0fa7d591754887d68faad0a",
  "txns": [
    {
      "disassembly": [
        "#pragma version 3",
        "intcblock 123",
        "arg_0",
        "btoi",
        "intc_0 // 123",
        "==",
        ""
      ],
      "logic-sig-messages": [
        "PASS"
      ],
      "logic-sig-trace": [
        {
          "line": 1,
          "pc": 1,
          "stack": []
        },
        {
          "line": 2,
          "pc": 4,
          "stack": []
        },
        {
          "line": 3,
          "pc": 5,
          "stack": [
            {
              "bytes": "AAAAAAAAAHs=",
              "type": 1,
              "uint": 0
            }


In [6]:
# This example is provided for informational purposes only and has not been audited for security.

from pyteal import *

def approval_program():
    on_creation = Seq([
        App.globalPut(Bytes("Creator"), Txn.sender()),
        Assert(Txn.application_args.length() == Int(4)),
        App.globalPut(Bytes("RegBegin"), Btoi(Txn.application_args[0])),
        App.globalPut(Bytes("RegEnd"), Btoi(Txn.application_args[1])),
        App.globalPut(Bytes("VoteBegin"), Btoi(Txn.application_args[2])),
        App.globalPut(Bytes("VoteEnd"), Btoi(Txn.application_args[3])),
        Return(Int(1))
    ])

    is_creator = Txn.sender() == App.globalGet(Bytes("Creator"))

    get_vote_of_sender = App.localGetEx(Int(0), App.id(), Bytes("voted"))

    on_closeout = Seq([
        get_vote_of_sender,
        If(And(Global.round() <= App.globalGet(Bytes("VoteEnd")), get_vote_of_sender.hasValue()),
            App.globalPut(get_vote_of_sender.value(), App.globalGet(get_vote_of_sender.value()) - Int(1))
        ),
        Return(Int(1))
    ])

    on_register = Return(And(
        Global.round() >= App.globalGet(Bytes("RegBegin")),
        Global.round() <= App.globalGet(Bytes("RegEnd"))
    ))

    choice = Txn.application_args[1]
    choice_tally = App.globalGet(choice)
    on_vote = Seq([
        Assert(And(
            Global.round() >= App.globalGet(Bytes("VoteBegin")),
            Global.round() <= App.globalGet(Bytes("VoteEnd"))
        )),
        get_vote_of_sender,
        If(get_vote_of_sender.hasValue(),
            Return(Int(0))
        ),
        App.globalPut(choice, choice_tally + Int(1)),
        App.localPut(Int(0), Bytes("voted"), choice),
        Return(Int(1))
    ])

    program = Cond(
        [Txn.application_id() == Int(0), on_creation],
        [Txn.on_completion() == OnComplete.DeleteApplication, Return(is_creator)],
        [Txn.on_completion() == OnComplete.UpdateApplication, Return(is_creator)],
        [Txn.on_completion() == OnComplete.CloseOut, on_closeout],
        [Txn.on_completion() == OnComplete.OptIn, on_register],
        [Txn.application_args[0] == Bytes("vote"), on_vote]
    )

    return program

def clear_state_program():
    get_vote_of_sender = App.localGetEx(Int(0), App.id(), Bytes("voted"))
    program = Seq([
        get_vote_of_sender,
        If(And(Global.round() <= App.globalGet(Bytes("VoteEnd")), get_vote_of_sender.hasValue()),
            App.globalPut(get_vote_of_sender.value(), App.globalGet(get_vote_of_sender.value()) - Int(1))
        ),
        Return(Int(1))
    ])

    return program

if __name__ == "__main__":
    with open('vote_approval.teal', 'w') as f:
        compiled = compileTeal(approval_program(), mode=Mode.Application, version=3)
        f.write(compiled)

    with open('vote_clear_state.teal', 'w') as f:
        compiled = compileTeal(clear_state_program(), mode=Mode.Application, version=3)
        f.write(compiled)

In [30]:
# based off https://github.com/algorand/docs/blob/cdf11d48a4b1168752e6ccaf77c8b9e8e599713a/examples/smart_contracts/v2/python/stateful_smart_contracts.py

import base64
import datetime

from algosdk.future import transaction
from algosdk import account, mnemonic
from algosdk.v2client import algod
from pyteal import compileTeal, Mode
# from vote import approval_program, clear_state_program

# user declared account mnemonics
# creator_mnemonic = "Your 25-word mnemonic goes here"
creator_mnemonic = "hero camp fold mushroom panic orphan lonely until rebel thumb bundle wrist jump key clay travel hello lift mule hamster square into caught abstract practice"

# user_mnemonic = "A second distinct 25-word mnemonic goes here"
user_mnemonic = "violin setup volume match vacuum jeans indoor taxi kangaroo pen kit picture skirt cloud alcohol shoot illegal hen chase breeze audit resist hire about broccoli"


# user declared algod connection parameters. Node must have EnableDeveloperAPI set to true in its config
# Create an algod client with node
algod_token = '25f90f9852eba8650750cb2e41b8dba807071d091168d9ece5554cabdaf5ce68'
# algod_address = 'https://testnet-algorand.api.purestake.io/ps2'
algod_address = 'http://localhost:8101'

#  Create an algod client with PureStake
# algod_token = 'VWZ0eDSeM6LIZX24KiiO4YyRTBpbeRF7x1YBUxZc'
# algod_address = 'https://testnet-algorand.api.purestake.io/ps2'
# purestake_token = {'X-Api-key': algod_token}
# algod_client = algod.AlgodClient(algod_token, algod_address, headers=purestake_token)


# helper function to compile program source
def compile_program(client, source_code):
    compile_response = client.compile(source_code)
    return base64.b64decode(compile_response['result'])

# helper function that converts a mnemonic passphrase into a private signing key
def get_private_key_from_mnemonic(mn):
    private_key = mnemonic.to_private_key(mn)
    return private_key

# helper function that waits for a given txid to be confirmed by the network
def wait_for_confirmation(client, txid):
    last_round = client.status().get('last-round')
    txinfo = client.pending_transaction_info(txid)
    while not (txinfo.get('confirmed-round') and txinfo.get('confirmed-round') > 0):
        print("Waiting for confirmation...")
        last_round += 1
        client.status_after_block(last_round)
        txinfo = client.pending_transaction_info(txid)
    print("Transaction {} confirmed in round {}.".format(txid, txinfo.get('confirmed-round')))
    return txinfo

def wait_for_round(client, round):
    last_round = client.status().get('last-round')
    print(f"Waiting for round {round}")
    while last_round < round:
        last_round += 1
        client.status_after_block(last_round)
        print(f"Round {last_round}")

# create new application
def create_app(client, private_key, approval_program, clear_program, global_schema, local_schema, app_args):
    # define sender as creator
    sender = account.address_from_private_key(private_key)

    # declare on_complete as NoOp
    on_complete = transaction.OnComplete.NoOpOC.real

	# get node suggested parameters
    params = client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 0

    # create unsigned transaction
    txn = transaction.ApplicationCreateTxn(sender, params, on_complete, \
                                            approval_program, clear_program, \
                                            global_schema, local_schema, app_args)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # await confirmation
    wait_for_confirmation(client, tx_id)

    # display results
    transaction_response = client.pending_transaction_info(tx_id)
    app_id = transaction_response['application-index']
    print("Created new app-id:", app_id)

    return app_id

# opt-in to application
def opt_in_app(client, private_key, index):
    # declare sender
    sender = account.address_from_private_key(private_key)
    print("OptIn from account: ",sender)

	# get node suggested parameters
    params = client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 0

    # create unsigned transaction
    txn = transaction.ApplicationOptInTxn(sender, params, index)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # await confirmation
    wait_for_confirmation(client, tx_id)

    # display results
    transaction_response = client.pending_transaction_info(tx_id)
    print("OptIn to app-id:", transaction_response['txn']['txn']['apid'])    

# call application
def call_app(client, private_key, index, app_args):
    # declare sender
    sender = account.address_from_private_key(private_key)
    print("Call from account:", sender)

	# get node suggested parameters
    params = client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 1000

    # create unsigned transaction
    txn = transaction.ApplicationNoOpTxn(sender, params, index, app_args)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # await confirmation
    wait_for_confirmation(client, tx_id)

def format_state(state):
    formatted = {}
    for item in state:
        key = item['key']
        value = item['value']
        formatted_key = base64.b64decode(key).decode('utf-8')
        if value['type'] == 1:
            # byte string
            if formatted_key == 'voted':
                formatted_value = base64.b64decode(value['bytes']).decode('utf-8')
            else:
                formatted_value = value['bytes']
            formatted[formatted_key] = formatted_value
        else:
            # integer
            formatted[formatted_key] = value['uint']
    return formatted

# read user local state
def read_local_state(client, addr, app_id):
    results = client.account_info(addr)
    for local_state in results['apps-local-state']:
        if local_state['id'] == app_id:
            if 'key-value' not in local_state:
                return {}
            return format_state(local_state['key-value'])
    return {}

# read app global state
def read_global_state(client, addr, app_id):
    results = client.account_info(addr)
    apps_created = results['created-apps']
    for app in apps_created:
        if app['id'] == app_id:
            return format_state(app['params']['global-state'])
    return {}

# delete application
def delete_app(client, private_key, index):
    # declare sender
    sender = account.address_from_private_key(private_key)

	# get node suggested parameters
    params = client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 0

    # create unsigned transaction
    txn = transaction.ApplicationDeleteTxn(sender, params, index)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # await confirmation
    wait_for_confirmation(client, tx_id)

    # display results
    transaction_response = client.pending_transaction_info(tx_id)
    print("Deleted app-id:", transaction_response['txn']['txn']['apid'])    

# close out from application
def close_out_app(client, private_key, index):
    # declare sender
    sender = account.address_from_private_key(private_key)

	# get node suggested parameters
    params = client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 1000

    # create unsigned transaction
    txn = transaction.ApplicationCloseOutTxn(sender, params, index)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # await confirmation
    wait_for_confirmation(client, tx_id)

    # display results
    transaction_response = client.pending_transaction_info(tx_id)
    print("Closed out from app-id: ",transaction_response['txn']['txn']['apid'])

# clear application
def clear_app(client, private_key, index):
    # declare sender
    sender = account.address_from_private_key(private_key)

	# get node suggested parameters
    params = client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 1000

    # create unsigned transaction
    txn = transaction.ApplicationClearStateTxn(sender, params, index)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # await confirmation
    wait_for_confirmation(client, tx_id)

    # display results
    transaction_response = client.pending_transaction_info(tx_id)
    print("Cleared app-id:", transaction_response['txn']['txn']['apid'])    

# convert 64 bit integer i to byte string
def intToBytes(i):
    return i.to_bytes(8, "big")

def main():
    # initialize an algodClient
    algod_client = algod.AlgodClient(algod_token, algod_address)

    # define private keys
    creator_private_key = get_private_key_from_mnemonic(creator_mnemonic)
    user_private_key = get_private_key_from_mnemonic(user_mnemonic)

    # declare application state storage (immutable)
    local_ints = 0
    local_bytes = 1
    global_ints = 24 # 4 for setup + 20 for choices. Use a larger number for more choices.
    global_bytes = 1
    global_schema = transaction.StateSchema(global_ints, global_bytes)
    local_schema = transaction.StateSchema(local_ints, local_bytes)

    # get PyTeal approval program
    approval_program_ast = approval_program()
    # compile program to TEAL assembly
    approval_program_teal = compileTeal(approval_program_ast, mode=Mode.Application, version=2)
    # compile program to binary
    approval_program_compiled = compile_program(algod_client, approval_program_teal)

    # get PyTeal clear state program
    clear_state_program_ast = clear_state_program()
    # compile program to TEAL assembly
    clear_state_program_teal = compileTeal(clear_state_program_ast, mode=Mode.Application, version=2)
    # compile program to binary
    clear_state_program_compiled = compile_program(algod_client, clear_state_program_teal)

    # configure registration and voting period
    status = algod_client.status()
    regBegin = status['last-round'] + 10
    regEnd = regBegin + 10
    voteBegin = regEnd + 1
    voteEnd = voteBegin + 10

    print(f"Registration rounds: {regBegin} to {regEnd}")
    print(f"Vote rounds: {voteBegin} to {voteEnd}")

    # create list of bytes for app args
    app_args = [
        intToBytes(regBegin),
        intToBytes(regEnd),
        intToBytes(voteBegin),
        intToBytes(voteEnd)
    ]

    # create new application
    app_id = create_app(algod_client, creator_private_key, approval_program_compiled, clear_state_program_compiled, global_schema, local_schema, app_args)

    # read global state of application
    print("Global state:", read_global_state(algod_client, account.address_from_private_key(creator_private_key), app_id))

    # wait for registration period to start
    wait_for_round(algod_client, regBegin)

    # opt-in to application
    opt_in_app(algod_client, user_private_key, app_id)

    wait_for_round(algod_client, voteBegin)

    # call application without arguments
    call_app(algod_client, user_private_key, app_id, [b'vote', b'choiceA'])

    # read local state of application from user account
    print("Local state:", read_local_state(algod_client, account.address_from_private_key(user_private_key), app_id))

    # wait for registration period to start
    wait_for_round(algod_client, voteEnd)

    # read global state of application
    global_state = read_global_state(algod_client, account.address_from_private_key(creator_private_key), app_id)
    print("Global state:", global_state)

    max_votes = 0
    max_votes_choice = None
    for key,value in global_state.items():
        if key not in ('RegBegin', 'RegEnd', 'VoteBegin', 'VoteEnd', 'Creator') and isinstance(value, int):
            if value > max_votes:
                max_votes = value
                max_votes_choice = key
    
    print("The winner is:", max_votes_choice)

    # delete application
    delete_app(algod_client, creator_private_key, app_id)

    # clear application from user account
    clear_app(algod_client, user_private_key, app_id)

if __name__ == "__main__":
    main()

AlgodHTTPError: {"message":"Forbidden"}

### Working samplearg.teal Code

In [1]:
import json
import base64

from algosdk.v2client import algod
from algosdk import account, mnemonic
from algosdk.v2client.models import DryrunRequest, DryrunSource
from algosdk.future.transaction import PaymentTxn, LogicSig, LogicSigTransaction
# from algosdk.testing import dryrun

def wait_for_confirmation(client, txid):
    """
    Utility function to wait until the transaction is
    confirmed before proceeding.
    """
    last_round = client.status().get('last-round')
    txinfo = client.pending_transaction_info(txid)
    while not (txinfo.get('confirmed-round') and txinfo.get('confirmed-round') > 0):
        print("Waiting for confirmation")
        last_round += 1
        client.status_after_block(last_round)
        txinfo = client.pending_transaction_info(txid)
    print("Transaction {} confirmed in round {}.".format(
        txid, txinfo.get('confirmed-round')))
    return txinfo

# dryrun source if provided, else dryrun compiled
def dryrun_debug(lstx, mysource):
    sources = []
    if (mysource != None):
        # source
        sources = [DryrunSource(field_name="lsig", source=mysource, txn_index=0)]
    drr = DryrunRequest(txns=[lstx], sources=sources)
    dryrun_response = algod_client.dryrun(drr)
    return dryrun_response

In [ ]:
try:
    receiver = "2WYBIISKSIIIF7TEDHB57JRZ2WWGPYG5GYVL74Y7KQLKLUFHNOZXAMZTBQ"
    
    # Create an algod client with node
    algod_token = '25f90f9852eba8650750cb2e41b8dba807071d091168d9ece5554cabdaf5ce68'
    algod_address = 'http://localhost:8101'
    algod_client = algod.AlgodClient(algod_token, algod_address)
    
    # Create an algod client with PureStake
    algod_token = 'VWZ0eDSeM6LIZX24KiiO4YyRTBpbeRF7x1YBUxZc'
    algod_address = 'https://testnet-algorand.api.purestake.io/ps2'
    purestake_token = {'X-Api-key': algod_token}
    algod_client = algod.AlgodClient(algod_token, algod_address, headers=purestake_token)

    myprogram = "samplearg.teal"

    # Read TEAL program
    data = open(myprogram, 'r').read()

    # Compile TEAL program
    # // This code is meant for learning purposes only	
    # // It should not be used in production	
    # // samplearg.teal
    # arg_0	
    # btoi
    # int 123	
    # ==	

    # // bto1	
    # // Opcode: 0x17	
    # // Pops: ... stack, []byte	
    # // Pushes: uint64	
    # // converts bytes X as big endian to uint64	
    # // btoi panics if the input is longer than 8 bytes

    response = algod_client.compile(data)
    # Print(response)
    print("Response Result = ", response['result'])
    print("Response Hash = ", response['hash'])

    # Create logic sig
    programstr = response['result']
    t = programstr.encode()
    program = base64.decodebytes(t)
    print(program)
    
    # Create arg to pass
    # string parameter
    # arg_str = "<my string>"
    # arg1 = arg_str.encode()
    # lsig = transaction.LogicSig(program, args=[arg1])

    # integer parameter
    # arg1 = (123).to_bytes(8, 'big')
    # lsig = transaction.LogicSig(program, args=[arg1])
    # see more info here: https://developer.algorand.org/docs/features/asc1/sdks/#accessing-teal-program-from-sdks

    # if TEAL program requires an arg,
    # if not, omit args param on LogicSig
    # lsig = LogicSig(program)
    arg1 = (123).to_bytes(8, 'big')
    lsig = LogicSig(program, args=[arg1])

    # Recover the account that is wanting to delegate signature
    passphrase = "hero camp fold mushroom panic orphan lonely until rebel thumb bundle wrist jump key clay travel hello lift mule hamster square into caught abstract practice"
    sk = mnemonic.to_private_key(passphrase)
    addr = account.address_from_private_key(sk)
    print("Address of Sender/Delegator: " + addr)

    # Sign the logic signature with an account sk
    lsig.sign(sk)

    # Get suggested parameters
    params = algod_client.suggested_params()
    # Comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 0

    # Build transaction
    amount = 0
    closeremainderto = None

    # Create a transaction
    txn = PaymentTxn(
        addr, params, receiver, amount, closeremainderto)
    # Create the LogicSigTransaction with contract account LogicSig
    lstx = LogicSigTransaction(txn, lsig)
    # transaction.write_to_file([lstx], "./simple.stxn")

    # compile
    dryrun_response_compiled = dryrun_debug(lstx, None)
    print ("COMPILED Dryrun results...")
    print(json.dumps(dryrun_response_compiled, indent=2))
    
    # source   
#     dryrun_respone_source = dryrun_debug(lstx, source)
#     print("SOURCE Dryrun results...")
#     print(json.dumps(dryrun_respone_source, indent=2))
   
    # Send raw LogicSigTransaction to network

    txid = algod_client.send_transaction(lstx)
    print("Transaction ID: " + txid)
    wait_for_confirmation(algod_client, txid)
except Exception as e:
    print("=== Exception ===")
    print(e)

In [4]:
# This example is provided for informational purposes only and has not been audited for security.

from pyteal import *

"""Atomic Swap"""

alice = Addr("VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U")
bob = Addr("FFVTKNLA4J47EHONVR3U247LFLHE3TNATDEJLDRI2E73FF52BGVK7MFWHY")
secret = Bytes("base32", "2323232323232323")
timeout = 3000

def htlc(tmpl_seller=alice,
         tmpl_buyer=bob,
         tmpl_fee=0,
         tmpl_secret=secret,
         tmpl_hash_fn=Sha256,
         tmpl_timeout=timeout):
    
    fee_cond = Txn.fee() < Int(tmpl_fee)
    safety_cond = And(
        Txn.type_enum() == TxnType.Payment,
        Txn.close_remainder_to() == Global.zero_address(),
        Txn.rekey_to() == Global.zero_address(),
    )

    recv_cond = And(
        Txn.receiver() == tmpl_seller,
        tmpl_hash_fn(Arg(0)) == tmpl_secret
    )
    
    esc_cond = And(
        Txn.receiver() == tmpl_buyer,
        Txn.first_valid() > Int(tmpl_timeout)
    )

    return And(
        fee_cond,
        safety_cond,
        Or(recv_cond, esc_cond)
    )

# Write to a file
with open('htlc.teal', 'w') as f:
    compiled = compileTeal(htlc(), mode=Mode.Signature, version=3)
    f.write(compiled)
    
try:
    receiver = "VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U"
    # Create an algod client with PureStake
    algod_token = 'VWZ0eDSeM6LIZX24KiiO4YyRTBpbeRF7x1YBUxZc'
    algod_address = 'https://testnet-algorand.api.purestake.io/ps2'
    purestake_token = {'X-Api-key': algod_token}
    algod_client = algod.AlgodClient(algod_token, algod_address, headers=purestake_token)

    myprogram = "htlc.teal"

    # Read TEAL program
    data = open(myprogram, 'r').read()
    response = algod_client.compile(data)
    # Print(response)
    print("Response Result = ", response['result'])
    print("Response Hash = ", response['hash'])

    # Create logic sig
    programstr = response['result']
    t = programstr.encode()
    program = base64.decodebytes(t)
    print(program)
    
    arg1 = (2323232323232323).to_bytes(8, 'big')
    lsig = LogicSig(program, args=[arg1])

    # Recover the account that is wanting to delegate signature
    passphrase = "hero camp fold mushroom panic orphan lonely until rebel thumb bundle wrist jump key clay travel hello lift mule hamster square into caught abstract practice"
    sk = mnemonic.to_private_key(passphrase)
    addr = account.address_from_private_key(sk)
    print("Address of Sender/Delegator: " + addr)

    # Sign the logic signature with an account sk
    lsig.sign(sk)

    # Get suggested parameters
    params = algod_client.suggested_params()
    # Comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 0

    # Build transaction
    amount = 0
    closeremainderto = None

    # Create a transaction
    txn = PaymentTxn(
        addr, params, receiver, amount, closeremainderto)
    # Create the LogicSigTransaction with contract account LogicSig
    lstx = LogicSigTransaction(txn, lsig)
    # transaction.write_to_file([lstx], "./simple.stxn")

    # compile
    dryrun_response_compiled = dryrun_debug(lstx, None)
    print ("COMPILED Dryrun results...")
    print(json.dumps(dryrun_response_compiled, indent=2))
    
    # source   
#     dryrun_respone_source = dryrun_debug(lstx, source)
#     print("SOURCE Dryrun results...")
#     print(json.dumps(dryrun_respone_source, indent=2))
   
    # Send raw LogicSigTransaction to network

    txid = algod_client.send_transaction(lstx)
    print("Transaction ID: " + txid)
    wait_for_confirmation(algod_client, txid)
except Exception as e:
    print("=== Exception ===")
    print(e)

Response Result =  AyADAAG4FyYDIK12+hJFveeUhYsLsN52wu+jwKSLrmRVgHgeIKJDHyPNCtb1vW9b1vW9b1sgKWs1NWDiefIdzax3TXPrKs5NzaCYyJWOKNE/spe6CaoxASIMMRAjEjEJMgMSEDEgMgMSEBAxBygSLQEpEhAxByoSMQIkDRAREA==
Response Hash =  JY5Q6WFZGD2BTURPO7QZOJN62V6GRUQOAJJQCQEVGXBJXO22KBQQ2ON6GY
b'\x03 \x03\x00\x01\xb8\x17&\x03 \xadv\xfa\x12E\xbd\xe7\x94\x85\x8b\x0b\xb0\xdev\xc2\xef\xa3\xc0\xa4\x8b\xaedU\x80x\x1e \xa2C\x1f#\xcd\n\xd6\xf5\xbdo[\xd6\xf5\xbdo[ )k55`\xe2y\xf2\x1d\xcd\xacwMs\xeb*\xceM\xcd\xa0\x98\xc8\x95\x8e(\xd1?\xb2\x97\xba\t\xaa1\x01"\x0c1\x10#\x121\t2\x03\x12\x101 2\x03\x12\x10\x101\x07(\x12-\x01)\x12\x101\x07*\x121\x02$\r\x10\x11\x10'
Address of Sender/Delegator: VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U
COMPILED Dryrun results...
{
  "error": "",
  "protocol-version": "https://github.com/algorandfoundation/specs/tree/65b4ab3266c52c56a0fa7d591754887d68faad0a",
  "txns": [
    {
      "disassembly": [
        "#pragma version 3",
        "intcblock 0 1 3000",
        "bytecblock 0x

=== Exception ===
{"message":"transaction {_struct:{} Sig:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] Msig:{_struct:{} Version:0 Threshold:0 Subsigs:[]} Lsig:{_struct:{} Logic:[3 32 3 0 1 184 23 38 3 32 173 118 250 18 69 189 231 148 133 139 11 176 222 118 194 239 163 192 164 139 174 100 85 128 120 30 32 162 67 31 35 205 10 214 245 189 111 91 214 245 189 111 91 32 41 107 53 53 96 226 121 242 29 205 172 119 77 115 235 42 206 77 205 160 152 200 149 142 40 209 63 178 151 186 9 170 49 1 34 12 49 16 35 18 49 9 50 3 18 16 49 32 50 3 18 16 16 49 7 40 18 45 1 41 18 16 49 7 42 18 49 2 36 13 16 17 16] Sig:[134 167 133 44 122 121 85 95 246 194 167 15 250 231 84 189 84 165 158 129 77 32 183 99 12 2 114 143 40 68 13 25 13 59 166 99 49 156 215 107 4 244 217 138 179 129 129 218 76 215 18 172 79 16 4 73 26 3 157 217 204 206 132 2] Msig:{_struct:{} Version:0 Threshold:0 Subsigs:[]} Args:[[0 8 64 247 173 91 98 67]]} Txn

## Working atomic transaction

In [2]:
import json
import time
import base64
import os
from algosdk.v2client import algod
from algosdk import mnemonic
from algosdk.future import transaction
from algosdk import encoding
from algosdk import account

# This atomic transfer example code requires three (3) acounts:
#  - account_1 requires a user-defined mnemonic and be funded with 1001000 microAlgos
#  - account_2 requires a user-defined mnemonic and be funded with 2001000 microAlgos
#  - account_3 auto-generated within the code, 1000000 microAlgos will be transfered here
# For account_1 and account_2, replcace the string "Your 25-word mnemonic goes here" in the code below.
# For account_3, ensure you note the mnemonic generated for future.
# Faucents available for funding accounts:
#  - TestNet: https://developer.algorand.org/docs/reference/algorand-networks/testnet/#faucet
#  - BetaNet: https://developer.algorand.org/docs/reference/algorand-networks/betanet/#faucet
# Replace the algod_address and algod_token parameters below to connect to your API host.

# user declared account mnemonics for account1 and account2
mnemonic_1 = "hero camp fold mushroom panic orphan lonely until rebel thumb bundle wrist jump key clay travel hello lift mule hamster square into caught abstract practice"
mnemonic_2 = "violin setup volume match vacuum jeans indoor taxi kangaroo pen kit picture skirt cloud alcohol shoot illegal hen chase breeze audit resist hire about broccoli"
# user declared algod connection parameters
# algod_address = "http://localhost:4001"
# algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"


# Create an algod client with PureStake
algod_token = 'VWZ0eDSeM6LIZX24KiiO4YyRTBpbeRF7x1YBUxZc'
algod_address = 'https://testnet-algorand.api.purestake.io/ps2'
purestake_token = {'X-Api-key': algod_token}
# algod_client = algod.AlgodClient(algod_token, algod_address, headers=purestake_token)

# Function that waits for a given txId to be confirmed by the network
def wait_for_confirmation(client, txid):
    last_round = client.status().get('last-round')
    txinfo = client.pending_transaction_info(txid)
    while not (txinfo.get('confirmed-round') and txinfo.get('confirmed-round') > 0):
        print("Waiting for confirmation...")
        last_round += 1
        client.status_after_block(last_round)
        txinfo = client.pending_transaction_info(txid)
    print("Transaction {} confirmed in round {}.".format(txid, txinfo.get('confirmed-round')))
    return txinfo


# utility function to get address string
def get_address(mn) :
    pk_account_a = mnemonic.to_private_key(mn)
    address = account.address_from_private_key(pk_account_a)
    print("Address :", address)
    return address


# utility function to generate new account
# def generate_new_account() :
# 	private_key, address = account.generate_account()
# 	print("Created new account: ", address)
# 	print("Generated mnemonic: \"{}\"".format(mnemonic.from_private_key(private_key)))
# 	return address    


# utility function to display account balance
def display_account_algo_balance(client, address) :
    account_info = client.account_info(address)
    print("{}: {} microAlgos".format(address, account_info["amount"]))


def group_transactions() :
	# Initialize an algodClient
    algod_client = algod.AlgodClient(algod_token, algod_address, headers=purestake_token)

	# declared account1 and account2 based on user supplied mnemonics
    print("Loading two existing accounts...")
    account_1 = get_address(mnemonic_1)
    account_2 = get_address(mnemonic_2)

	# convert mnemonic1 and mnemonic2 using the mnemonic.ToPrivateKey() helper function
    sk_1 = mnemonic.to_private_key(mnemonic_1)
    sk_2 = mnemonic.to_private_key(mnemonic_2)

	# generate account3, display mnemonic, wait
	# print("Generating new account...")
#     account_3 = generate_new_account()
    account_3 = "2WYBIISKSIIIF7TEDHB57JRZ2WWGPYG5GYVL74Y7KQLKLUFHNOZXAMZTBQ"
    print("!! NOTICE !! Please retain the above generated \"25-word mnemonic passphrase\" for future use.")

	# display account balances
    print("Initial balances:")
    display_account_algo_balance(algod_client, account_1)
    display_account_algo_balance(algod_client, account_2)
    display_account_algo_balance(algod_client, account_3)

	# get node suggested parameters
    params = algod_client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 0

	# create transactions
    print("Creating transactions...")
	# from account 1 to account 3
    sender = account_1
    receiver = account_3
    amount = 0
    txn_1 = transaction.PaymentTxn(sender, params, receiver, amount)
    print("...txn_1: from {} to {} for {} microAlgos".format(sender, receiver, amount))
    print("...created txn_1: ", txn_1.get_txid())

	# from account 2 to account 1
    sender = account_2
    receiver = account_1
    amount = 0
    txn_2 = transaction.PaymentTxn(sender, params, receiver, amount)
    print("...txn_2: from {} to {} for {} microAlgos".format(sender, receiver, amount))
    print("...created txn_2: ", txn_2.get_txid())

	# combine transations
    print("Combining transactions...")
	# the SDK does this implicitly within grouping below
     
    print("Grouping transactions...")
	# compute group id and put it into each transaction
    group_id = transaction.calculate_group_id([txn_1, txn_2])
    print("...computed groupId: ", group_id)
    txn_1.group = group_id
    txn_2.group = group_id

	# split transaction group
    print("Splitting unsigned transaction group...")
    # this example does not use files on disk, so splitting is implicit above

	# sign transactions
    print("Signing transactions...")
    stxn_1 = txn_1.sign(sk_1)    
    print("...account1 signed txn_1: ", stxn_1.get_txid())
    stxn_2 = txn_2.sign(sk_2)
    print("...account2 signed txn_2: ", stxn_2.get_txid())

	# assemble transaction group
    print("Assembling transaction group...")
    signed_group =  [stxn_1, stxn_2]

	# send transactions
    print("Sending transaction group...")
    tx_id = algod_client.send_transactions(signed_group)

    # wait for confirmation
    wait_for_confirmation(algod_client, tx_id) 

	# display account balances
    print("Final balances:")
    display_account_algo_balance(algod_client, account_1)
    display_account_algo_balance(algod_client, account_2)
    display_account_algo_balance(algod_client, account_3)

	# display confirmed transaction group
	# tx1
    confirmed_txn = algod_client.pending_transaction_info(txn_1.get_txid())
    print("Transaction information: {}".format(json.dumps(confirmed_txn, indent=4)))

	# tx2
    confirmed_txn = algod_client.pending_transaction_info(txn_2.get_txid())
    print("Transaction information: {}".format(json.dumps(confirmed_txn, indent=4)))


group_transactions()

Loading two existing accounts...
Address : VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U
Address : FFVTKNLA4J47EHONVR3U247LFLHE3TNATDEJLDRI2E73FF52BGVK7MFWHY
!! NOTICE !! Please retain the above generated "25-word mnemonic passphrase" for future use.
Initial balances:
VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U: 8535596 microAlgos
FFVTKNLA4J47EHONVR3U247LFLHE3TNATDEJLDRI2E73FF52BGVK7MFWHY: 7894010 microAlgos
2WYBIISKSIIIF7TEDHB57JRZ2WWGPYG5GYVL74Y7KQLKLUFHNOZXAMZTBQ: 9183010 microAlgos
Creating transactions...
...txn_1: from VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U to 2WYBIISKSIIIF7TEDHB57JRZ2WWGPYG5GYVL74Y7KQLKLUFHNOZXAMZTBQ for 0 microAlgos
...created txn_1:  2ZRCSOTTDYDJ4WSHYPOWKE6EGPOGZYD4QUN2BBLPQ57RIP5PIFZA
...txn_2: from FFVTKNLA4J47EHONVR3U247LFLHE3TNATDEJLDRI2E73FF52BGVK7MFWHY to VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U for 0 microAlgos
...created txn_2:  OXZDFR5UXH2X2OONVYAHVYECJMQI4WCWANKYZOWXMJTJWUJPK7JQ
Combin

## Accounts

Account 1
VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U



Account 2
FFVTKNLA4J47EHONVR3U247LFLHE3TNATDEJLDRI2E73FF52BGVK7MFWHY


Account 3
2WYBIISKSIIIF7TEDHB57JRZ2WWGPYG5GYVL74Y7KQLKLUFHNOZXAMZTBQ

Copy off accounts above and add TestNet Algo funds using the TestNet Dispenser at https://bank.testnet.algorand.network/
copy off the following mnemonic code for use later



mnemonic1 = "hero camp fold mushroom panic orphan lonely until rebel thumb bundle wrist jump key clay travel hello lift mule hamster square into caught abstract practice"


mnemonic2 = "violin setup volume match vacuum jeans indoor taxi kangaroo pen kit picture skirt cloud alcohol shoot illegal hen chase breeze audit resist hire about broccoli"


mnemonic3 = "sing access small blast neither surprise awkward october catch survey own kit diesel emerge kangaroo unhappy client volume advice silly decrease worry august abandon giant"

# Statefull Smart Contract with Node

#### The term "stateful" means the application is able to store information or "maintain state" within the ledger. 

### https://developer.algorand.org/docs/features/asc1/stateful/hello_world/#application-components


#### Every Algorand stateful application is comprised of at least two programs: approval and clear state. 


#### Approval Program

The approval program is the main application logic for the smart contract.
#### Clear State Program

The clear state program is used to retire an application for specific users. This will clear the smart contract's local state variables from the user's balance record.
#### State Storage Locations

Applications have access to three storage locations for stateful data: global, local, and external.


#### Global
Global data are stored within the application itself. The program will read from and write to its global storage the same regardless of which account submitted the application call transaction.


#### Local
Each program may read from and write to local storage within the account object of the "calling account". This allows an application developer to store user-specific data and vary program execution logic at run-time per user.

#### External
Each program may read both the global and local state storage locations for a limited number of other external programs and accounts. For more information on external programs and accounts see the stateful smart contract documentation.

#### State Data

Both global and local data are stored as key/value pairs, where the key is bytes and the value may be either a bytes or uint64.
```json
{ 
  "key": {
    "type": <1 || 2>,
    "value": <[]byte || uint64>
  }
}
```


### https://developer.algorand.org/docs/features/asc1/stateful/sdks/#declarations
#### All stateful applications are comprised of state storage, an approval program and a clear program. Details of each are found within the stateful smart contract guide.

### https://developer.algorand.org/docs/features/asc1/stateful/hello_world/#draft-application-code

#### write following in `approval_program.teal` file

```bash
#pragma version 4

// read global state
byte "counter"
dup
app_global_get

// increment the value
int 1
+

// store to scratch space
dup
store 0

// update global state
app_global_put

// load return value as approval
load 0
return




```

#### write following in `clear_state_program.teal` file

```bash
#pragma version 4
// This program clears program state

int 1
```


#### write following in a bash file eg. `app_create.sh` file
```bash
# the app creater address i.e the algrand account public key 
export ADDR_CREATOR="VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U"

# the approval TEAL Program file for the app
export TEAL_APPROVAL_PROG="approval_program.teal"

# the clear TEAL Program file for the app
export TEAL_CLEAR_PROG="clear_state_program.teal"

# some global state, the value may be either a bytes or uint64
export GLOBAL_BYTESLICES=0
export GLOBAL_INTS=1

# some local state,the value may be either a bytes or uint64
export LOCAL_BYTESLICES=0
export LOCAL_INTS=0

goal app create --creator $ADDR_CREATOR \
                --approval-prog "$TEAL_APPROVAL_PROG" \
                --clear-prog "$TEAL_CLEAR_PROG" \
                --global-byteslices $GLOBAL_BYTESLICES \
                --global-ints $GLOBAL_INTS \
                --local-byteslices $LOCAL_BYTESLICES \
                --local-ints $LOCAL_INTS 
```

#### more about goal https://developer.algorand.org/docs/reference/cli/goal/app/app/


#### now open terminal (or git bash if you are on window) and run the bash file

`
./app_create.sh
`

#### create a wallet if you get this message No wallets found. Create a new wallet with `goal wallet new [wallet name]`

### creating a  wallet  `goal wallet new wallet1`

### if you see the error: Couldn't sign tx with kmd: key does not exist in this wallet

### check the wallet via `goal wallet list` and account list via `goal account list`, also run the kmd via `goal kmd start`

### if it show there is no account then you can either create or import an account (to import an account you need the menomic of the account)

###### https://developer.algorand.org/docs/reference/cli/goal/account/import/
### import account using menomic `goal account import -m "hero camp fold mushroom panic orphan lonely until rebel thumb bundle wrist jump key clay travel hello lift mule hamster square into caught abstract practice"`

### check the wallet via `goal wallet list` and account list via `goal account list`
#### if everything work fine then run 
`
./app_create.sh
`

### still error check for catchup
##### TestNet https://algorand-catchpoints.s3.us-east-2.amazonaws.com/channel/testnet/latest.catchpoint
`goal node catchup <value_from_above-site>`
### watchout node status for every 5 second and wait for `Sync Time:` to become zero(0)
`goal node status -w 5000` 

```bash
goal app create --creator {ACCOUNT} --approval-prog ./crowd_fund.teal --global-byteslices 3 --global-ints 5 --local-byteslices 0 --local-ints 1 --app-arg “int:begindatetimestamp” --app-arg "int:enddatetimespamp" --app-arg "int:1000000" --app-arg "addr:"{ACCOUNT} --app-arg "int:fundclosedatetimestamp" --clear-prog ./crowd_fund_close.teal -d ~/node/data
```
```bash
goal app create --creator VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U --approval-prog approval_program.teal --clear-prog clear_state_program.teal --global-byteslices 0 --global-ints 1 --local-byteslices 0 --local-ints 0
```

```bash
goal app create --creator "VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U" --approval-prog "./approval_program.teal" --clear-prog "./clear_state_program.teal" --global-byteslices 0 --global-ints 1 --local-byteslices 0 --local-ints 0
```



goal account new -w wallet1 -f
Please enter the password for wallet 'wallet1': 
Created new account with address 7ZBCRDDBMRQ7ZBV5RLFUEXLEZC3JY3I65QWRDRJZAISDJE73F6UUHZ5HWE

goal account addpartkey -a 7ZBCRDDBMRQ7ZBV5RLFUEXLEZC3JY3I65QWRDRJZAISDJE73F6UUHZ5HWE --roundFirstValid=900000 --roundLastValid=15250995
### 

#### Couldn't broadcast tx with algod: HTTP 400 Bad Request: TransactionPool.Remember: TransactionPool.ingest: no pending block evaluator


# Statefull Smart Contract with py-algorand-sdk

In [4]:

# Function that waits for a given txId to be confirmed by the network
def wait_for_confirmation(client, txid):
    last_round = client.status().get('last-round')
    txinfo = client.pending_transaction_info(txid)
    while not (txinfo.get('confirmed-round') and txinfo.get('confirmed-round') > 0):
        print("Waiting for confirmation...")
        last_round += 1
        client.status_after_block(last_round)
        txinfo = client.pending_transaction_info(txid)
    print("Transaction {} confirmed in round {}.".format(txid, txinfo.get('confirmed-round')))
    return txinfo


# utility function to get address string
def get_address(mn) :
    pk_account_a = mnemonic.to_private_key(mn)
    address = account.address_from_private_key(pk_account_a)
    print("Address :", address)
    return address


# utility function to generate new account
# def generate_new_account() :
# 	private_key, address = account.generate_account()
# 	print("Created new account: ", address)
# 	print("Generated mnemonic: \"{}\"".format(mnemonic.from_private_key(private_key)))
# 	return address    


# utility function to display account balance
def display_account_algo_balance(client, address) :
    account_info = client.account_info(address)
    print("{}: {} microAlgos".format(address, account_info["amount"]))



In [3]:

# helper function to compile program source
def compile_program(client, source_code):
    compile_response = client.compile(source_code)
    return base64.b64decode(compile_response['result'])

# helper function that converts a mnemonic passphrase into a private signing key
def get_private_key_from_mnemonic(mn):
    private_key = mnemonic.to_private_key(mn)
    return private_key

# helper function that waits for a given txid to be confirmed by the network
def wait_for_confirmation(client, txid):
    last_round = client.status().get('last-round')
    txinfo = client.pending_transaction_info(txid)
    while not (txinfo.get('confirmed-round') and txinfo.get('confirmed-round') > 0):
        print("Waiting for confirmation...")
        last_round += 1
        client.status_after_block(last_round)
        txinfo = client.pending_transaction_info(txid)
    print("Transaction {} confirmed in round {}.".format(txid, txinfo.get('confirmed-round')))
    return txinfo

def wait_for_round(client, round):
    last_round = client.status().get('last-round')
    print(f"Waiting for round {round}")
    while last_round < round:
        last_round += 1
        client.status_after_block(last_round)
        print(f"Round {last_round}")

# create new application
def create_app(client, private_key, approval_program, clear_program, global_schema, local_schema, app_args):
    # define sender as creator
    sender = account.address_from_private_key(private_key)

    # declare on_complete as NoOp
    on_complete = transaction.OnComplete.NoOpOC.real

	# get node suggested parameters
    params = client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 0

    # create unsigned transaction
    txn = transaction.ApplicationCreateTxn(sender, params, on_complete, \
                                            approval_program, clear_program, \
                                            global_schema, local_schema, app_args)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # await confirmation
    wait_for_confirmation(client, tx_id)

    # display results
    transaction_response = client.pending_transaction_info(tx_id)
    app_id = transaction_response['application-index']
    print("Created new app-id:", app_id)

    return app_id

# opt-in to application
def opt_in_app(client, private_key, index):
    # declare sender
    sender = account.address_from_private_key(private_key)
    print("OptIn from account: ",sender)

	# get node suggested parameters
    params = client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 0

    # create unsigned transaction
    txn = transaction.ApplicationOptInTxn(sender, params, index)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # await confirmation
    wait_for_confirmation(client, tx_id)

    # display results
    transaction_response = client.pending_transaction_info(tx_id)
    print("OptIn to app-id:", transaction_response['txn']['txn']['apid'])    

# call application
def call_app(client, private_key, index, app_args):
    # declare sender
    sender = account.address_from_private_key(private_key)
    print("Call from account:", sender)

	# get node suggested parameters
    params = client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 1000

    # create unsigned transaction
    txn = transaction.ApplicationNoOpTxn(sender, params, index, app_args)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # await confirmation
    wait_for_confirmation(client, tx_id)

def format_state(state):
    formatted = {}
    for item in state:
        key = item['key']
        value = item['value']
        formatted_key = base64.b64decode(key).decode('utf-8')
        if value['type'] == 1:
            # byte string
            if formatted_key == 'voted':
                formatted_value = base64.b64decode(value['bytes']).decode('utf-8')
            else:
                formatted_value = value['bytes']
            formatted[formatted_key] = formatted_value
        else:
            # integer
            formatted[formatted_key] = value['uint']
    return formatted

# read user local state
def read_local_state(client, addr, app_id):
    results = client.account_info(addr)
    for local_state in results['apps-local-state']:
        if local_state['id'] == app_id:
            if 'key-value' not in local_state:
                return {}
            return format_state(local_state['key-value'])
    return {}

# read app global state
def read_global_state(client, addr, app_id):
    results = client.account_info(addr)
    apps_created = results['created-apps']
    for app in apps_created:
        if app['id'] == app_id:
            return format_state(app['params']['global-state'])
    return {}

# delete application
def delete_app(client, private_key, index):
    # declare sender
    sender = account.address_from_private_key(private_key)

	# get node suggested parameters
    params = client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 0

    # create unsigned transaction
    txn = transaction.ApplicationDeleteTxn(sender, params, index)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # await confirmation
    wait_for_confirmation(client, tx_id)

    # display results
    transaction_response = client.pending_transaction_info(tx_id)
    print("Deleted app-id:", transaction_response['txn']['txn']['apid'])    

# close out from application
def close_out_app(client, private_key, index):
    # declare sender
    sender = account.address_from_private_key(private_key)

	# get node suggested parameters
    params = client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 1000

    # create unsigned transaction
    txn = transaction.ApplicationCloseOutTxn(sender, params, index)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # await confirmation
    wait_for_confirmation(client, tx_id)

    # display results
    transaction_response = client.pending_transaction_info(tx_id)
    print("Closed out from app-id: ",transaction_response['txn']['txn']['apid'])

# clear application
def clear_app(client, private_key, index):
    # declare sender
    sender = account.address_from_private_key(private_key)

	# get node suggested parameters
    params = client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 1000

    # create unsigned transaction
    txn = transaction.ApplicationClearStateTxn(sender, params, index)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # await confirmation
    wait_for_confirmation(client, tx_id)

    # display results
    transaction_response = client.pending_transaction_info(tx_id)
    print("Cleared app-id:", transaction_response['txn']['txn']['apid'])    

# convert 64 bit integer i to byte string
def intToBytes(i):
    return i.to_bytes(8, "big")


In [26]:
import json
import time
import base64
import os
from algosdk.v2client import algod
from algosdk import mnemonic
from algosdk.future import transaction
from algosdk import encoding
from algosdk import account


# user declared account mnemonics
creator_mnemonic = "hero camp fold mushroom panic orphan lonely until rebel thumb bundle wrist jump key clay travel hello lift mule hamster square into caught abstract practice"
user_mnemonic = "violin setup volume match vacuum jeans indoor taxi kangaroo pen kit picture skirt cloud alcohol shoot illegal hen chase breeze audit resist hire about broccoli"

# user declared algod connection parameters
# algod_address = "http://localhost:4001"
# algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"
# algod_client = algod.AlgodClient(algod_token, algod_address)
algod_token = 'VWZ0eDSeM6LIZX24KiiO4YyRTBpbeRF7x1YBUxZc'
algod_address = 'https://testnet-algorand.api.purestake.io/ps2'
purestake_token = {'X-Api-key': algod_token}
algod_client = algod.AlgodClient(algod_token, algod_address, headers=purestake_token)

# declare application state storage (immutable)
local_ints = 1
local_bytes = 1
global_ints = 1
global_bytes = 0

# define schema
global_schema = transaction.StateSchema(global_ints, global_bytes)
local_schema = transaction.StateSchema(local_ints, local_bytes)


# declare clear state program source
clear_program_source = """#pragma version 4
int 1
"""

# declare approval program source
approval_program_source = """#pragma version 4
// Handle each possible OnCompletion type. We don't have to worry about
// handling ClearState, because the ClearStateProgram will execute in that
// case, not the ApprovalProgram.

txn OnCompletion
int NoOp
==
bnz handle_noop

txn OnCompletion
int OptIn
==
bnz handle_optin

txn OnCompletion
int CloseOut
==
bnz handle_closeout

txn OnCompletion
int UpdateApplication
==
bnz handle_updateapp

txn OnCompletion
int DeleteApplication
==
bnz handle_deleteapp

// Unexpected OnCompletion value. Should be unreachable.
err

handle_noop:
// Handle NoOp
// Check for creator
addr VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U
txn Sender
==
bnz handle_optin

// read global state
byte "counter"
dup
app_global_get

// increment the value
int 1
+

// store to scratch space
dup
store 0

// update global state
app_global_put

// read local state for sender
int 0
byte "counter"
app_local_get

// increment the value
int 1
+
store 1

// update local state for sender
int 0
byte "counter"
load 1
app_local_put

// load return value as approval
load 0
return

handle_optin:
// Handle OptIn
// approval
int 1
return

handle_closeout:
// Handle CloseOut
//approval
int 1
return

handle_deleteapp:
// Check for creator
addr VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U
txn Sender
==
return

handle_updateapp:
// Check for creator
addr VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U
txn Sender
==
return"""

# get node suggested parameters
params = algod_client.suggested_params()
# comment out the next two (2) lines to use suggested fees
params.flat_fee = True
params.fee = 0


# declare on_complete as NoOp
on_complete = transaction.OnComplete.NoOpOC.real

# helper function to compile program source
def compile_program(client, source_code) :
    compile_response = client.compile(source_code.decode('utf-8'))
    return base64.b64decode(compile_response['result'])

In [ ]:
# declare sender

private_key = mnemonic.to_private_key(creator_mnemonic)
sender = account.address_from_private_key(private_key)

# with open("approval_program_source.teal", "rb") as f:
#     approval_program_source_file = f.read()
    
# with open("clear_program_source.teal", "rb") as f:
#     clear_program_source_file = f.read()
    
approval_program_source_file = "approval_program_source.teal"
clear_program_source_file = "clear_program_source.teal"

def compile_program(client, source_code):
    compile_response = client.compile(source_code)
    return base64.b64decode(compile_response['result'])


approval_program = compile_program(algod_client,approval_program_source)
clear_program = compile_program(algod_client,clear_program_source)

# create unsigned transaction
txn = transaction.ApplicationCreateTxn(sender, params, on_complete, \
                                        approval_program, clear_program, \
                                        global_schema, local_schema)

# sign transaction
signed_txn = txn.sign(private_key)
tx_id = signed_txn.transaction.get_txid()

# send transaction
algod_client.send_transactions([signed_txn])

# await confirmation
wait_for_confirmation(algod_client, tx_id)

In [36]:
# display results
transaction_response = algod_client.pending_transaction_info(tx_id)
app_id = transaction_response['application-index']
print("Created new app-id: ",app_id)

Created new app-id:  20709490


In [35]:
# sign, send, await
from pprint import pprint
# display results
transaction_response = algod_client.pending_transaction_info(tx_id)
pprint(transaction_response)

{'application-index': 20709490,
 'confirmed-round': 15649119,
 'pool-error': '',
 'txn': {'sig': 'xpgnU8eHBU0hIWlMyQV3mz8pfO3R058knAH2NnTO6IkgISx8HtkHWpVlwg3rilAFhqA/+Y1gwJPMGbXzFnwXDQ==',
         'txn': {'apap': 'BCACAQAmAiCtdvoSRb3nlIWLC7DedsLvo8Cki65kVYB4HiCiQx8jzQdjb3VudGVyMRkjEkAAIDEZIhJAADgxGYECEkAAMjEZgQQSQAAxMRmBBRJAACQAKDEAEkAAGClJZCIISTUAZyMpYiIINQEjKTQBZjQAQyJDIkMoMQASQygxABJD',
                 'apgs': {'nui': 1},
                 'apls': {'nbs': 1, 'nui': 1},
                 'apsu': 'BIEB',
                 'fee': 1000,
                 'fv': 15649116,
                 'gen': 'testnet-v1.0',
                 'gh': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=',
                 'lv': 15650116,
                 'snd': 'VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U',
                 'type': 'appl'}}}


In [30]:
read_local_state(algod_client,)

KeyError: 'apid'

In [38]:
# create unsigned transaction
index = app_id
txn = transaction.ApplicationNoOpTxn(sender, params, index)

# sign, send, await 

# display results
transaction_response = algod_client.pending_transaction_info(tx_id)
print("Called app-id: ",transaction_response['txn']['txn']['apid'])
if "global-state-delta" in transaction_response :
    print("Global State updated :\n",transaction_response['global-state-delta'])
if "local-state-delta" in transaction_response :
    print("Local State updated :\n",transaction_response['local-state-delta'])


KeyError: 'apid'

In [22]:
# read user local state
def read_local_state(client, addr, app_id) :   
    results = client.account_info(addr)
    local_state = results['apps-local-state'][0]
    for index in local_state :
        if local_state[index] == app_id :
            print(f"local_state of account {addr} for app_id {app_id}: ", local_state['key-value'])

# read app global state
def read_global_state(client, addr, app_id) :   
    results = client.account_info(addr)
    apps_created = results['created-apps']
    for app in apps_created :
        if app['id'] == app_id :
            print(f"global_state for app_id {app_id}: ", app['params']['global-state'])
            
            
            
# create unsigned transaction
txn = transaction.ApplicationUpdateTxn(sender, params, app_id, \
                                        approval_program, clear_program, app_args)

# sign, send, await 

# display results
transaction_response = algod_client.pending_transaction_info(tx_id)
app_id = transaction_response['txn']['txn']['apid']
print("Updated existing app-id: ",app_id)



# call application with arguments
now = datetime.datetime.now().strftime("%H:%M:%S")
app_args = [now.encode("utf-8")]

# create unsigned transaction
txn = transaction.ApplicationNoOpTxn(sender, params, index, app_args)

# sign, send, await 

# display results
transaction_response = algod_client.pending_transaction_info(tx_id)
print("Called app-id: ",transaction_response['txn']['txn']['apid'])
if "global-state-delta" in transaction_response :
    print("Global State updated :\n",transaction_response['global-state-delta'])
if "local-state-delta" in transaction_response :
    print("Local State updated :\n",transaction_response['local-state-delta'])
    
    
# create unsigned transaction
txn = transaction.ApplicationCloseOutTxn(sender, params, index)

# sign, send, await 

# display results
transaction_response = algod_client.pending_transaction_info(tx_id)
print("Closed out from app-id: ",transaction_response['txn']['txn']['apid'])


# create unsigned transaction
txn = transaction.ApplicationDeleteTxn(sender, params, index)

# sign, send, await

# display results
transaction_response = algod_client.pending_transaction_info(tx_id)
print("Deleted app-id: ",transaction_response['txn']['txn']['apid'])


# create unsigned transaction
txn = transaction.ApplicationClearStateTxn(sender, params, index)

# sign, send, await 

# display results
transaction_response = algod_client.pending_transaction_info(tx_id)
print("Cleared app-id: ",transaction_response['txn']['txn']['apid']) 

AlgodHTTPError: {"message":"could not find the transaction in the transaction pool or in the last 1000 confirmed rounds"}


In [50]:
# based off https://github.com/algorand/docs/blob/cdf11d48a4b1168752e6ccaf77c8b9e8e599713a/examples/smart_contracts/v2/python/stateful_smart_contracts.py

import base64
import datetime

from algosdk.future import transaction
from algosdk import account, mnemonic
from algosdk.v2client import algod
from pyteal import compileTeal, Mode
# from vote import approval_program, clear_state_program

# user declared account mnemonics
# creator_mnemonic = "Your 25-word mnemonic goes here"
creator_mnemonic = "hero camp fold mushroom panic orphan lonely until rebel thumb bundle wrist jump key clay travel hello lift mule hamster square into caught abstract practice"

# user_mnemonic = "A second distinct 25-word mnemonic goes here"
user_mnemonic = "violin setup volume match vacuum jeans indoor taxi kangaroo pen kit picture skirt cloud alcohol shoot illegal hen chase breeze audit resist hire about broccoli"

In [51]:

# declare clear state program source
clear_state_program = """#pragma version 4
int 1
"""

# declare approval program source
approval_program = """#pragma version 4
// Handle each possible OnCompletion type. We don't have to worry about
// handling ClearState, because the ClearStateProgram will execute in that
// case, not the ApprovalProgram.

txn OnCompletion
int NoOp
==
bnz handle_noop

txn OnCompletion
int OptIn
==
bnz handle_optin

txn OnCompletion
int CloseOut
==
bnz handle_closeout

txn OnCompletion
int UpdateApplication
==
bnz handle_updateapp

txn OnCompletion
int DeleteApplication
==
bnz handle_deleteapp

// Unexpected OnCompletion value. Should be unreachable.
err

handle_noop:
// Handle NoOp
// Check for creator
addr VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U
txn Sender
==
bnz handle_optin

// read global state
byte "counter"
dup
app_global_get

// increment the value
int 1
+

// store to scratch space
dup
store 0

// update global state
app_global_put

// read local state for sender
int 0
byte "counter"
app_local_get

// increment the value
int 1
+
store 1

// update local state for sender
int 0
byte "counter"
load 1
app_local_put

// load return value as approval
load 0
return

handle_optin:
// Handle OptIn
// approval
int 1
return

handle_closeout:
// Handle CloseOut
//approval
int 1
return

handle_deleteapp:
// Check for creator
addr VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U
txn Sender
==
return

handle_updateapp:
// Check for creator
addr VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U
txn Sender
==
return"""

In [53]:
# user declared algod connection parameters. Node must have EnableDeveloperAPI set to true in its config
# Create an algod client with node
# algod_token = '25f90f9852eba8650750cb2e41b8dba807071d091168d9ece5554cabdaf5ce68'
# algod_address = 'https://testnet-algorand.api.purestake.io/ps2'
# algod_address = 'http://localhost:8101'

#  Create an algod client with PureStake
# algod_token = 'VWZ0eDSeM6LIZX24KiiO4YyRTBpbeRF7x1YBUxZc'
# algod_address = 'https://testnet-algorand.api.purestake.io/ps2'
# purestake_token = {'X-Api-key': algod_token}
# algod_client = algod.AlgodClient(algod_token, algod_address, headers=purestake_token)
algod_token = 'VWZ0eDSeM6LIZX24KiiO4YyRTBpbeRF7x1YBUxZc'
algod_address = 'https://testnet-algorand.api.purestake.io/ps2'
purestake_token = {'X-Api-key': algod_token}
# algod_client = algod.AlgodClient(algod_token, algod_address, headers=purestake_token)

# helper function to compile program source
def compile_program(client, source_code):
    compile_response = client.compile(source_code)
    return base64.b64decode(compile_response['result'])

# helper function that converts a mnemonic passphrase into a private signing key
def get_private_key_from_mnemonic(mn):
    private_key = mnemonic.to_private_key(mn)
    return private_key

# helper function that waits for a given txid to be confirmed by the network
def wait_for_confirmation(client, txid):
    last_round = client.status().get('last-round')
    txinfo = client.pending_transaction_info(txid)
    while not (txinfo.get('confirmed-round') and txinfo.get('confirmed-round') > 0):
        print("Waiting for confirmation...")
        last_round += 1
        client.status_after_block(last_round)
        txinfo = client.pending_transaction_info(txid)
    print("Transaction {} confirmed in round {}.".format(txid, txinfo.get('confirmed-round')))
    return txinfo

def wait_for_round(client, round):
    last_round = client.status().get('last-round')
    print(f"Waiting for round {round}")
    while last_round < round:
        last_round += 1
        client.status_after_block(last_round)
        print(f"Round {last_round}")

# create new application
def create_app(client, private_key, approval_program, clear_program, global_schema, local_schema, app_args):
    # define sender as creator
    sender = account.address_from_private_key(private_key)

    # declare on_complete as NoOp
    on_complete = transaction.OnComplete.NoOpOC.real

	# get node suggested parameters
    params = client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 0

    # create unsigned transaction
    txn = transaction.ApplicationCreateTxn(sender, params, on_complete, \
                                            approval_program, clear_program, \
                                            global_schema, local_schema, app_args)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # await confirmation
    wait_for_confirmation(client, tx_id)

    # display results
    transaction_response = client.pending_transaction_info(tx_id)
    app_id = transaction_response['application-index']
    print("Created new app-id:", app_id)

    return app_id

# opt-in to application
def opt_in_app(client, private_key, index):
    # declare sender
    sender = account.address_from_private_key(private_key)
    print("OptIn from account: ",sender)

	# get node suggested parameters
    params = client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 0

    # create unsigned transaction
    txn = transaction.ApplicationOptInTxn(sender, params, index)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # await confirmation
    wait_for_confirmation(client, tx_id)

    # display results
    transaction_response = client.pending_transaction_info(tx_id)
    print("OptIn to app-id:", transaction_response['txn']['txn']['apid'])    

# call application
def call_app(client, private_key, index, app_args):
    # declare sender
    sender = account.address_from_private_key(private_key)
    print("Call from account:", sender)

	# get node suggested parameters
    params = client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 1000

    # create unsigned transaction
    txn = transaction.ApplicationNoOpTxn(sender, params, index, app_args)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # await confirmation
    wait_for_confirmation(client, tx_id)

def format_state(state):
    formatted = {}
    for item in state:
        key = item['key']
        value = item['value']
        formatted_key = base64.b64decode(key).decode('utf-8')
        if value['type'] == 1:
            # byte string
            if formatted_key == 'voted':
                formatted_value = base64.b64decode(value['bytes']).decode('utf-8')
            else:
                formatted_value = value['bytes']
            formatted[formatted_key] = formatted_value
        else:
            # integer
            formatted[formatted_key] = value['uint']
    return formatted

# read user local state
def read_local_state(client, addr, app_id):
    results = client.account_info(addr)
    for local_state in results['apps-local-state']:
        if local_state['id'] == app_id:
            if 'key-value' not in local_state:
                return {}
            return format_state(local_state['key-value'])
    return {}

# read app global state
def read_global_state(client, addr, app_id):
    results = client.account_info(addr)
    apps_created = results['created-apps']
    for app in apps_created:
        if app['id'] == app_id:
            return format_state(app['params']['global-state'])
    return {}

# delete application
def delete_app(client, private_key, index):
    # declare sender
    sender = account.address_from_private_key(private_key)

	# get node suggested parameters
    params = client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 0

    # create unsigned transaction
    txn = transaction.ApplicationDeleteTxn(sender, params, index)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # await confirmation
    wait_for_confirmation(client, tx_id)

    # display results
    transaction_response = client.pending_transaction_info(tx_id)
    print("Deleted app-id:", transaction_response['txn']['txn']['apid'])    

# close out from application
def close_out_app(client, private_key, index):
    # declare sender
    sender = account.address_from_private_key(private_key)

	# get node suggested parameters
    params = client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 1000

    # create unsigned transaction
    txn = transaction.ApplicationCloseOutTxn(sender, params, index)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # await confirmation
    wait_for_confirmation(client, tx_id)

    # display results
    transaction_response = client.pending_transaction_info(tx_id)
    print("Closed out from app-id: ",transaction_response['txn']['txn']['apid'])

# clear application
def clear_app(client, private_key, index):
    # declare sender
    sender = account.address_from_private_key(private_key)

	# get node suggested parameters
    params = client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 1000

    # create unsigned transaction
    txn = transaction.ApplicationClearStateTxn(sender, params, index)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # await confirmation
    wait_for_confirmation(client, tx_id)

    # display results
    transaction_response = client.pending_transaction_info(tx_id)
    print("Cleared app-id:", transaction_response['txn']['txn']['apid'])    

# convert 64 bit integer i to byte string
def intToBytes(i):
    return i.to_bytes(8, "big")

def main():
    # initialize an algodClient
    algod_client = algod.AlgodClient(algod_token, algod_address, headers=purestake_token)

    # define private keys
    creator_private_key = get_private_key_from_mnemonic(creator_mnemonic)
    user_private_key = get_private_key_from_mnemonic(user_mnemonic)

    # declare application state storage (immutable)
    local_ints = 0
    local_bytes = 1
    global_ints = 24 # 4 for setup + 20 for choices. Use a larger number for more choices.
    global_bytes = 1
    global_schema = transaction.StateSchema(global_ints, global_bytes)
    local_schema = transaction.StateSchema(local_ints, local_bytes)

#     # get PyTeal approval program
#     approval_program_ast = approval_program()
#     # compile program to TEAL assembly
#     approval_program_teal = compileTeal(approval_program_ast, mode=Mode.Application, version=2)
#     # compile program to binary
#     approval_program_compiled = compile_program(algod_client, approval_program_teal)

#     # get PyTeal clear state program
#     clear_state_program_ast = clear_state_program()
#     # compile program to TEAL assembly
#     clear_state_program_teal = compileTeal(clear_state_program_ast, mode=Mode.Application, version=2)
#     # compile program to binary
#     clear_state_program_compiled = compile_program(algod_client, clear_state_program_teal)

    
    # get PyTeal approval program
    # compile program to binary
    approval_program_compiled = compile_program(algod_client, approval_program)

    # compile program to binary
    clear_state_program_compiled = compile_program(algod_client, clear_state_program)
    # configure registration and voting period
    status = algod_client.status()
    regBegin = status['last-round'] + 10
    regEnd = regBegin + 10
    voteBegin = regEnd + 1
    voteEnd = voteBegin + 10

    print(f"Registration rounds: {regBegin} to {regEnd}")
    print(f"Vote rounds: {voteBegin} to {voteEnd}")

    # create list of bytes for app args
    app_args = [
        intToBytes(regBegin),
        intToBytes(regEnd),
        intToBytes(voteBegin),
        intToBytes(voteEnd)
    ]

    # create new application
    app_id = create_app(algod_client, creator_private_key, approval_program_compiled, clear_state_program_compiled, global_schema, local_schema, app_args)

    # read global state of application
#     print("Global state:", read_global_state(algod_client, account.address_from_private_key(creator_private_key), app_id))

    # wait for registration period to start
    wait_for_round(algod_client, regBegin)

    # opt-in to application
    opt_in_app(algod_client, user_private_key, app_id)

    wait_for_round(algod_client, voteBegin)

    # call application without arguments
    call_app(algod_client, user_private_key, app_id, [b'vote', b'choiceA'])

    # read local state of application from user account
    print("Local state:", read_local_state(algod_client, account.address_from_private_key(user_private_key), app_id))

    # wait for registration period to start
    wait_for_round(algod_client, voteEnd)

    # read global state of application
    global_state = read_global_state(algod_client, account.address_from_private_key(creator_private_key), app_id)
    print("Global state:", global_state)

    max_votes = 0
    max_votes_choice = None
    for key,value in global_state.items():
        if key not in ('RegBegin', 'RegEnd', 'VoteBegin', 'VoteEnd', 'Creator') and isinstance(value, int):
            if value > max_votes:
                max_votes = value
                max_votes_choice = key
    
    print("The winner is:", max_votes_choice)

    # delete application
    delete_app(algod_client, creator_private_key, app_id)

    # clear application from user account
    clear_app(algod_client, user_private_key, app_id)

if __name__ == "__main__":
    main()

Registration rounds: 15649859 to 15649869
Vote rounds: 15649870 to 15649880
Waiting for confirmation...
Transaction 7I6TGMYDP5ZI5RHOODMUPHEWHFWPVUF3LLB3YEGO4EBYDWZVJBWA confirmed in round 15649852.
Created new app-id: 20712971
Waiting for round 15649859
Round 15649853
Round 15649854
Round 15649855
Round 15649856
Round 15649857
Round 15649858
Round 15649859
OptIn from account:  FFVTKNLA4J47EHONVR3U247LFLHE3TNATDEJLDRI2E73FF52BGVK7MFWHY
Waiting for confirmation...
Transaction PHAHN4JQG4WTARVQISNYNITVSWIRUJNE5OK5YKUTMRIA7C7NW5NQ confirmed in round 15649862.
OptIn to app-id: 20712971
Waiting for round 15649870
Round 15649863
Round 15649864
Round 15649865
Round 15649866
Round 15649867
Round 15649868
Round 15649869
Round 15649870
Call from account: FFVTKNLA4J47EHONVR3U247LFLHE3TNATDEJLDRI2E73FF52BGVK7MFWHY


AlgodHTTPError: {"message":"TransactionPool.Remember: transaction SROPYPVM4YPDO3324YL7ELON34BSMIEASCQCWCVPHRJXKRDC3XNQ: logic eval error: store integer count 1 exceeds schema integer count 0"}


In [2]:
# This example is provided for informational purposes only and has not been audited for security.

from pyteal import *

def approval_program():
    on_creation = Seq([
        App.globalPut(Bytes("Creator"), Txn.sender()),
        Assert(Txn.application_args.length() == Int(4)),
        App.globalPut(Bytes("RegBegin"), Btoi(Txn.application_args[0])),
        App.globalPut(Bytes("RegEnd"), Btoi(Txn.application_args[1])),
        App.globalPut(Bytes("VoteBegin"), Btoi(Txn.application_args[2])),
        App.globalPut(Bytes("VoteEnd"), Btoi(Txn.application_args[3])),
        Return(Int(1))
    ])

    is_creator = Txn.sender() == App.globalGet(Bytes("Creator"))

    get_vote_of_sender = App.localGetEx(Int(0), App.id(), Bytes("voted"))

    on_closeout = Seq([
        get_vote_of_sender,
        If(And(Global.round() <= App.globalGet(Bytes("VoteEnd")), get_vote_of_sender.hasValue()),
            App.globalPut(get_vote_of_sender.value(), App.globalGet(get_vote_of_sender.value()) - Int(1))
        ),
        Return(Int(1))
    ])
    
    on_register = Return(And(
        Global.round() >= App.globalGet(Bytes("RegBegin")),
        Global.round() <= App.globalGet(Bytes("RegEnd"))
    ))

    choice = Txn.application_args[1]
    choice_tally = App.globalGet(choice)
    on_vote = Seq([
        Assert(And(
            Global.round() >= App.globalGet(Bytes("VoteBegin")),
            Global.round() <= App.globalGet(Bytes("VoteEnd"))
        )),
        get_vote_of_sender,
        If(get_vote_of_sender.hasValue(),
            Return(Int(0))
        ),
        App.globalPut(choice, choice_tally + Int(1)),
        App.localPut(Int(0), Bytes("voted"), choice),
        Return(Int(1))
    ])

    program = Cond(
        [Txn.application_id() == Int(0), on_creation],
        [Txn.on_completion() == OnComplete.DeleteApplication, Return(is_creator)],
        [Txn.on_completion() == OnComplete.UpdateApplication, Return(is_creator)],
        [Txn.on_completion() == OnComplete.CloseOut, on_closeout],
        [Txn.on_completion() == OnComplete.OptIn, on_register],
        [Txn.application_args[0] == Bytes("vote"), on_vote]
    )

    return program

def clear_state_program():
    get_vote_of_sender = App.localGetEx(Int(0), App.id(), Bytes("voted"))
    program = Seq([
        get_vote_of_sender,
        If(And(Global.round() <= App.globalGet(Bytes("VoteEnd")), get_vote_of_sender.hasValue()),
            App.globalPut(get_vote_of_sender.value(), App.globalGet(get_vote_of_sender.value()) - Int(1))
        ),
        Return(Int(1))
    ])

    return program

if __name__ == "__main__":
    with open('vote_approval.teal', 'w') as f:
        compiled = compileTeal(approval_program(), mode=Mode.Application, version=3)
        print(compiled)
        f.write(compiled)

    with open('vote_clear_state.teal', 'w') as f:
        compiled = compileTeal(clear_state_program(), mode=Mode.Application, version=3)
        print(compiled)
        f.write(compiled)

#pragma version 3
txn ApplicationID
int 0
==
bnz l16
txn OnCompletion
int DeleteApplication
==
bnz l15
txn OnCompletion
int UpdateApplication
==
bnz l14
txn OnCompletion
int CloseOut
==
bnz l11
txn OnCompletion
int OptIn
==
bnz l10
txna ApplicationArgs 0
byte "vote"
==
bnz l7
err
l7:
global Round
byte "VoteBegin"
app_global_get
>=
global Round
byte "VoteEnd"
app_global_get
<=
&&
assert
int 0
global CurrentApplicationID
byte "voted"
app_local_get_ex
store 0
store 1
load 0
bz l9
int 0
return
l9:
txna ApplicationArgs 1
txna ApplicationArgs 1
app_global_get
int 1
+
app_global_put
int 0
byte "voted"
txna ApplicationArgs 1
app_local_put
int 1
return
l10:
global Round
byte "RegBegin"
app_global_get
>=
global Round
byte "RegEnd"
app_global_get
<=
&&
return
l11:
int 0
global CurrentApplicationID
byte "voted"
app_local_get_ex
store 0
store 1
global Round
byte "VoteEnd"
app_global_get
<=
load 0
&&
bz l13
load 1
load 1
app_global_get
int 1
-
app_global_put
l13:
int 1
return
l14:
txn Sender
byte "C

## PyTeal Examples

## Atomic Swap

In [2]:
# This example is provided for informational purposes only and has not been audited for security.

from pyteal import *

"""Atomic Swap"""

alice = Addr("VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U")
bob = Addr("FFVTKNLA4J47EHONVR3U247LFLHE3TNATDEJLDRI2E73FF52BGVK7MFWHY")
secret = Bytes("base32", "2323232323232323")
timeout = 3000

def htlc(tmpl_seller=alice,
         tmpl_buyer=bob,
         tmpl_fee=0,
         tmpl_secret=secret,
         tmpl_hash_fn=Sha256,
         tmpl_timeout=timeout):
    
    fee_cond = Txn.fee() < Int(tmpl_fee)
    safety_cond = And(
        Txn.type_enum() == TxnType.Payment,
        Txn.close_remainder_to() == Global.zero_address(),
        Txn.rekey_to() == Global.zero_address(),
    )

    recv_cond = And(
        Txn.receiver() == tmpl_seller,
        tmpl_hash_fn(Arg(0)) == tmpl_secret
    )
    
    esc_cond = And(
        Txn.receiver() == tmpl_buyer,
        Txn.first_valid() > Int(tmpl_timeout)
    )

    return And(
        fee_cond,
        safety_cond,
        Or(recv_cond, esc_cond)
    )

teal_file = compileTeal(htlc(), mode=Mode.Signature, version=3)
if __name__ == "__main__":
    print(compileTeal(htlc(), mode=Mode.Signature, version=3))
    
    


#pragma version 3
txn Fee
int 0
<
txn TypeEnum
int pay
==
txn CloseRemainderTo
global ZeroAddress
==
&&
txn RekeyTo
global ZeroAddress
==
&&
&&
txn Receiver
addr VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U
==
arg 0
sha256
byte base32(2323232323232323)
==
&&
txn Receiver
addr FFVTKNLA4J47EHONVR3U247LFLHE3TNATDEJLDRI2E73FF52BGVK7MFWHY
==
txn FirstValid
int 3000
>
&&
||
&&


In [18]:
import json
import time
import base64
import os
from algosdk.v2client import algod
from algosdk import mnemonic
from algosdk.future import transaction
from algosdk import encoding
from algosdk import account

from algosdk.v2client.models import DryrunRequest, DryrunSource
from algosdk.future.transaction import PaymentTxn, LogicSig, LogicSigTransaction
# from algosdk.testing import dryrun

# This atomic transfer example code requires three (3) acounts:
#  - account_1 requires a user-defined mnemonic and be funded with 1001000 microAlgos
#  - account_2 requires a user-defined mnemonic and be funded with 2001000 microAlgos
#  - account_3 auto-generated within the code, 1000000 microAlgos will be transfered here
# For account_1 and account_2, replcace the string "Your 25-word mnemonic goes here" in the code below.
# For account_3, ensure you note the mnemonic generated for future.
# Faucents available for funding accounts:
#  - TestNet: https://developer.algorand.org/docs/reference/algorand-networks/testnet/#faucet
#  - BetaNet: https://developer.algorand.org/docs/reference/algorand-networks/betanet/#faucet
# Replace the algod_address and algod_token parameters below to connect to your API host.

# user declared account mnemonics for account1 and account2
mnemonic_1 = "hero camp fold mushroom panic orphan lonely until rebel thumb bundle wrist jump key clay travel hello lift mule hamster square into caught abstract practice"
mnemonic_2 = "violin setup volume match vacuum jeans indoor taxi kangaroo pen kit picture skirt cloud alcohol shoot illegal hen chase breeze audit resist hire about broccoli"
# user declared algod connection parameters
# algod_address = "http://localhost:4001"
# algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"


# Create an algod client with PureStake
# algod_token = 'VWZ0eDSeM6LIZX24KiiO4YyRTBpbeRF7x1YBUxZc'
# algod_address = 'https://testnet-algorand.api.purestake.io/ps2'
# purestake_token = {'X-Api-key': algod_token}
# algod_client = algod.AlgodClient(algod_token, algod_address, headers=purestake_token)

# Function that waits for a given txId to be confirmed by the network
def wait_for_confirmation(client, txid):
    last_round = client.status().get('last-round')
    txinfo = client.pending_transaction_info(txid)
    while not (txinfo.get('confirmed-round') and txinfo.get('confirmed-round') > 0):
        print("Waiting for confirmation...")
        last_round += 1
        client.status_after_block(last_round)
        txinfo = client.pending_transaction_info(txid)
    print("Transaction {} confirmed in round {}.".format(txid, txinfo.get('confirmed-round')))
    return txinfo


# utility function to get address string
def get_address(mn) :
    pk_account_a = mnemonic.to_private_key(mn)
    address = account.address_from_private_key(pk_account_a)
    print("Address :", address)
    return address

# dryrun source if provided, else dryrun compiled
def dryrun_debug(algod_client, lstx, mysource):
    sources = []
    if (mysource != None):
        # source
        sources = [DryrunSource(field_name="lsig", source=mysource, txn_index=0)]
    drr = DryrunRequest(txns=[lstx], sources=sources)
    dryrun_response = algod_client.dryrun(drr)
    return dryrun_response
# utility function to generate new account
# def generate_new_account() :
# 	private_key, address = account.generate_account()
# 	print("Created new account: ", address)
# 	print("Generated mnemonic: \"{}\"".format(mnemonic.from_private_key(private_key)))
# 	return address    


# utility function to display account balance
def display_account_algo_balance(client, address) :
    account_info = client.account_info(address)
    print("{}: {} microAlgos".format(address, account_info["amount"]))


def atomic_transactions() :
        # Create an algod client with PureStake
    algod_token = 'VWZ0eDSeM6LIZX24KiiO4YyRTBpbeRF7x1YBUxZc'
    algod_address = 'https://testnet-algorand.api.purestake.io/ps2'
    purestake_token = {'X-Api-key': algod_token}
    algod_client = algod.AlgodClient(algod_token, algod_address, headers=purestake_token)
    try:
        # Initialize an algodClient
    #     algod_token = 'VWZ0eDSeM6LIZX24KiiO4YyRTBpbeRF7x1YBUxZc'
    #     algod_address = 'https://testnet-algorand.api.purestake.io/ps2'
    #     purestake_token = {'X-Api-key': algod_token}
    #     algod_client = algod.AlgodClient(algod_token, algod_address, headers=purestake_token)

        # declared account1 and account2 based on user supplied mnemonics
        print("Loading two existing accounts...")
        account_1 = get_address(mnemonic_1)
        account_2 = get_address(mnemonic_2)

        # convert mnemonic1 and mnemonic2 using the mnemonic.ToPrivateKey() helper function
        sk_1 = mnemonic.to_private_key(mnemonic_1)
        sk_2 = mnemonic.to_private_key(mnemonic_2)

        # generate account3, display mnemonic, wait
        # print("Generating new account...")
    #     account_3 = generate_new_account()
        account_3 = "2WYBIISKSIIIF7TEDHB57JRZ2WWGPYG5GYVL74Y7KQLKLUFHNOZXAMZTBQ"
        print("!! NOTICE !! Please retain the above generated \"25-word mnemonic passphrase\" for future use.")

        # display account balances
        print("Initial balances:")
        display_account_algo_balance(algod_client, account_1)
        display_account_algo_balance(algod_client, account_2)
        display_account_algo_balance(algod_client, account_3)

        # get node suggested parameters
        params = algod_client.suggested_params()
        # comment out the next two (2) lines to use suggested fees
        params.flat_fee = True
        params.fee = 0

    
        receiver = account_2




        # Read TEAL program
        data = teal_file

        # Compile TEAL program
        # // This code is meant for learning purposes only	
        # // It should not be used in production	
        # // samplearg.teal
        # arg_0	
        # btoi
        # int 123	
        # ==	

        # // bto1	
        # // Opcode: 0x17	
        # // Pops: ... stack, []byte	
        # // Pushes: uint64	
        # // converts bytes X as big endian to uint64	
        # // btoi panics if the input is longer than 8 bytes

        response = algod_client.compile(data)
        # Print(response)
        print("Response Result = ", response['result'])
        print("Response Hash = ", response['hash'])

        # Create logic sig
        programstr = response['result']
        t = programstr.encode()
        program = base64.decodebytes(t)
        print(program)

        # Create arg to pass
        # string parameter
        # arg_str = "<my string>"
        # arg1 = arg_str.encode()
        # lsig = transaction.LogicSig(program, args=[arg1])

        # integer parameter
        # arg1 = (123).to_bytes(8, 'big')
        # lsig = transaction.LogicSig(program, args=[arg1])
        # see more info here: https://developer.algorand.org/docs/features/asc1/sdks/#accessing-teal-program-from-sdks

        # if TEAL program requires an arg,
        # if not, omit args param on LogicSig
        # lsig = LogicSig(program)
        arg1 = (2323232323232323).to_bytes(8, 'big')
        lsig = LogicSig(program, args=[arg1])

        # Recover the account that is wanting to delegate signature
        passphrase = mnemonic_1
        sk = mnemonic.to_private_key(passphrase)
        addr = account.address_from_private_key(sk)
        print("Address of Sender/Delegator: " + addr)

        # Sign the logic signature with an account sk
        lsig.sign(sk)

        # Get suggested parameters
        params = algod_client.suggested_params()
        # Comment out the next two (2) lines to use suggested fees
        params.flat_fee = True
        params.fee = 0

        # Build transaction
        amount = 0
        closeremainderto = None

        # Create a transaction
        txn = PaymentTxn(
            addr, params, receiver, amount, closeremainderto)
        # Create the LogicSigTransaction with contract account LogicSig
        lstx = LogicSigTransaction(txn, lsig)
        # transaction.write_to_file([lstx], "./simple.stxn")

        # compile
        dryrun_response_compiled = dryrun_debug(algod_client,lstx, None)
        print ("COMPILED Dryrun results...")
        print(json.dumps(dryrun_response_compiled, indent=2))

        # source   
    #     dryrun_respone_source = dryrun_debug(lstx, source)
    #     print("SOURCE Dryrun results...")
    #     print(json.dumps(dryrun_respone_source, indent=2))

        # Send raw LogicSigTransaction to network

        txid = algod_client.send_transaction(lstx)
        print("Transaction ID: " + txid)
        wait_for_confirmation(algod_client, txid)
    except Exception as e:
        print("=== Exception ===")
        print(e)


atomic_transactions()

Loading two existing accounts...
Address : VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U
Address : FFVTKNLA4J47EHONVR3U247LFLHE3TNATDEJLDRI2E73FF52BGVK7MFWHY
!! NOTICE !! Please retain the above generated "25-word mnemonic passphrase" for future use.
Initial balances:
VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U: 18530596 microAlgos
FFVTKNLA4J47EHONVR3U247LFLHE3TNATDEJLDRI2E73FF52BGVK7MFWHY: 7892010 microAlgos
2WYBIISKSIIIF7TEDHB57JRZ2WWGPYG5GYVL74Y7KQLKLUFHNOZXAMZTBQ: 9183010 microAlgos
Response Result =  AyADAAG4FyYDIK12+hJFveeUhYsLsN52wu+jwKSLrmRVgHgeIKJDHyPNCtb1vW9b1vW9b1sgKWs1NWDiefIdzax3TXPrKs5NzaCYyJWOKNE/spe6CaoxASIMMRAjEjEJMgMSEDEgMgMSEBAxBygSLQEpEhAxByoSMQIkDRAREA==
Response Hash =  JY5Q6WFZGD2BTURPO7QZOJN62V6GRUQOAJJQCQEVGXBJXO22KBQQ2ON6GY
b'\x03 \x03\x00\x01\xb8\x17&\x03 \xadv\xfa\x12E\xbd\xe7\x94\x85\x8b\x0b\xb0\xdev\xc2\xef\xa3\xc0\xa4\x8b\xaedU\x80x\x1e \xa2C\x1f#\xcd\n\xd6\xf5\xbdo[\xd6\xf5\xbdo[ )k55`\xe2y\xf2\x1d\xcd\xacwMs\xeb*\xceM\xcd\xa0

=== Exception ===
{"message":"transaction {_struct:{} Sig:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] Msig:{_struct:{} Version:0 Threshold:0 Subsigs:[]} Lsig:{_struct:{} Logic:[3 32 3 0 1 184 23 38 3 32 173 118 250 18 69 189 231 148 133 139 11 176 222 118 194 239 163 192 164 139 174 100 85 128 120 30 32 162 67 31 35 205 10 214 245 189 111 91 214 245 189 111 91 32 41 107 53 53 96 226 121 242 29 205 172 119 77 115 235 42 206 77 205 160 152 200 149 142 40 209 63 178 151 186 9 170 49 1 34 12 49 16 35 18 49 9 50 3 18 16 49 32 50 3 18 16 16 49 7 40 18 45 1 41 18 16 49 7 42 18 49 2 36 13 16 17 16] Sig:[134 167 133 44 122 121 85 95 246 194 167 15 250 231 84 189 84 165 158 129 77 32 183 99 12 2 114 143 40 68 13 25 13 59 166 99 49 156 215 107 4 244 217 138 179 129 129 218 76 215 18 172 79 16 4 73 26 3 157 217 204 206 132 2] Msig:{_struct:{} Version:0 Threshold:0 Subsigs:[]} Args:[[0 8 64 247 173 91 98 67]]} Txn

## Application Mode

In [21]:
# This example is provided for informational purposes only and has not been audited for security.

from pyteal import *

def approval_program():
    on_creation = Seq([
        App.globalPut(Bytes("Creator"), Txn.sender()),
        Assert(Txn.application_args.length() == Int(4)),
        App.globalPut(Bytes("RegBegin"), Btoi(Txn.application_args[0])),
        App.globalPut(Bytes("RegEnd"), Btoi(Txn.application_args[1])),
        App.globalPut(Bytes("VoteBegin"), Btoi(Txn.application_args[2])),
        App.globalPut(Bytes("VoteEnd"), Btoi(Txn.application_args[3])),
        Return(Int(1))
    ])

    is_creator = Txn.sender() == App.globalGet(Bytes("Creator"))

    get_vote_of_sender = App.localGetEx(Int(0), App.id(), Bytes("voted"))

    on_closeout = Seq([
        get_vote_of_sender,
        If(And(Global.round() <= App.globalGet(Bytes("VoteEnd")), get_vote_of_sender.hasValue()),
            App.globalPut(get_vote_of_sender.value(), App.globalGet(get_vote_of_sender.value()) - Int(1))
        ),
        Return(Int(1))
    ])

    on_register = Return(And(
        Global.round() >= App.globalGet(Bytes("RegBegin")),
        Global.round() <= App.globalGet(Bytes("RegEnd"))
    ))

    choice = Txn.application_args[1]
    choice_tally = App.globalGet(choice)
    on_vote = Seq([
        Assert(And(
            Global.round() >= App.globalGet(Bytes("VoteBegin")),
            Global.round() <= App.globalGet(Bytes("VoteEnd"))
        )),
        get_vote_of_sender,
        If(get_vote_of_sender.hasValue(),
            Return(Int(0))
        ),
        App.globalPut(choice, choice_tally + Int(1)),
        App.localPut(Int(0), Bytes("voted"), choice),
        Return(Int(1))
    ])

    program = Cond(
        [Txn.application_id() == Int(0), on_creation],
        [Txn.on_completion() == OnComplete.DeleteApplication, Return(is_creator)],
        [Txn.on_completion() == OnComplete.UpdateApplication, Return(is_creator)],
        [Txn.on_completion() == OnComplete.CloseOut, on_closeout],
        [Txn.on_completion() == OnComplete.OptIn, on_register],
        [Txn.application_args[0] == Bytes("vote"), on_vote]
    )

    return program

def clear_state_program():
    get_vote_of_sender = App.localGetEx(Int(0), App.id(), Bytes("voted"))
    program = Seq([
        get_vote_of_sender,
        If(And(Global.round() <= App.globalGet(Bytes("VoteEnd")), get_vote_of_sender.hasValue()),
            App.globalPut(get_vote_of_sender.value(), App.globalGet(get_vote_of_sender.value()) - Int(1))
        ),
        Return(Int(1))
    ])

    return program

# if __name__ == "__main__":
#     def approval_program():
#         compiled = compileTeal(approval_program(), mode=Mode.Application, version=3)
#         return approval_program
#     def clear_state_program():
#         compiled = compileTeal(approval_program(), mode=Mode.Application, version=3)
#         return approval_program
#     with open('vote_approval.teal', 'w') as f:
#         compiled = compileTeal(approval_program(), mode=Mode.Application, version=2)
#         f.write(compiled)

#     with open('vote_clear_state.teal', 'w') as f:
#         compiled = compileTeal(clear_state_program(), mode=Mode.Application, version=2)
#         f.write(compiled)

In [27]:
# based off https://github.com/algorand/docs/blob/cdf11d48a4b1168752e6ccaf77c8b9e8e599713a/examples/smart_contracts/v2/python/stateful_smart_contracts.py

import base64
import datetime

from algosdk.future import transaction
from algosdk import account, mnemonic
from algosdk.v2client import algod
from pyteal import compileTeal, Mode
# from vote import approval_program, clear_state_program

# user declared account mnemonics
creator_mnemonic = "hero camp fold mushroom panic orphan lonely until rebel thumb bundle wrist jump key clay travel hello lift mule hamster square into caught abstract practice"
user_mnemonic =  "violin setup volume match vacuum jeans indoor taxi kangaroo pen kit picture skirt cloud alcohol shoot illegal hen chase breeze audit resist hire about broccoli"
mnemonic_1 = "hero camp fold mushroom panic orphan lonely until rebel thumb bundle wrist jump key clay travel hello lift mule hamster square into caught abstract practice"
mnemonic_2 = "violin setup volume match vacuum jeans indoor taxi kangaroo pen kit picture skirt cloud alcohol shoot illegal hen chase breeze audit resist hire about broccoli"
# user declared algod connection parameters. Node must have EnableDeveloperAPI set to true in its config
algod_token = 'VWZ0eDSeM6LIZX24KiiO4YyRTBpbeRF7x1YBUxZc'
algod_address = 'https://testnet-algorand.api.purestake.io/ps2'
purestake_token = {'X-Api-key': algod_token}
# algod_client = algod.AlgodClient(algod_token, algod_address, headers=purestake_token)

# helper function to compile program source
def compile_program(client, source_code):
    compile_response = client.compile(source_code)
    return base64.b64decode(compile_response['result'])

# helper function that converts a mnemonic passphrase into a private signing key
def get_private_key_from_mnemonic(mn):
    private_key = mnemonic.to_private_key(mn)
    return private_key

# helper function that waits for a given txid to be confirmed by the network
def wait_for_confirmation(client, txid):
    last_round = client.status().get('last-round')
    txinfo = client.pending_transaction_info(txid)
    while not (txinfo.get('confirmed-round') and txinfo.get('confirmed-round') > 0):
        print("Waiting for confirmation...")
        last_round += 1
        client.status_after_block(last_round)
        txinfo = client.pending_transaction_info(txid)
    print("Transaction {} confirmed in round {}.".format(txid, txinfo.get('confirmed-round')))
    return txinfo

def wait_for_round(client, round):
    last_round = client.status().get('last-round')
    print(f"Waiting for round {round}")
    while last_round < round:
        last_round += 1
        client.status_after_block(last_round)
        print(f"Round {last_round}")

# create new application
def create_app(client, private_key, approval_program, clear_program, global_schema, local_schema, app_args):
    # define sender as creator
    sender = account.address_from_private_key(private_key)

    # declare on_complete as NoOp
    on_complete = transaction.OnComplete.NoOpOC.real

	# get node suggested parameters
    params = client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 1000

    # create unsigned transaction
    txn = transaction.ApplicationCreateTxn(sender, params, on_complete, \
                                            approval_program, clear_program, \
                                            global_schema, local_schema, app_args)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # await confirmation
    wait_for_confirmation(client, tx_id)

    # display results
    transaction_response = client.pending_transaction_info(tx_id)
    app_id = transaction_response['application-index']
    print("Created new app-id:", app_id)

    return app_id

# opt-in to application
def opt_in_app(client, private_key, index):
    # declare sender
    sender = account.address_from_private_key(private_key)
    print("OptIn from account: ",sender)

	# get node suggested parameters
    params = client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 1000

    # create unsigned transaction
    txn = transaction.ApplicationOptInTxn(sender, params, index)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # await confirmation
    wait_for_confirmation(client, tx_id)

    # display results
    transaction_response = client.pending_transaction_info(tx_id)
    print("OptIn to app-id:", transaction_response['txn']['txn']['apid'])    

# call application
def call_app(client, private_key, index, app_args):
    # declare sender
    sender = account.address_from_private_key(private_key)
    print("Call from account:", sender)

	# get node suggested parameters
    params = client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 1000

    # create unsigned transaction
    txn = transaction.ApplicationNoOpTxn(sender, params, index, app_args)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # await confirmation
    wait_for_confirmation(client, tx_id)

def format_state(state):
    formatted = {}
    for item in state:
        key = item['key']
        value = item['value']
        formatted_key = base64.b64decode(key).decode('utf-8')
        if value['type'] == 1:
            # byte string
            if formatted_key == 'voted':
                formatted_value = base64.b64decode(value['bytes']).decode('utf-8')
            else:
                formatted_value = value['bytes']
            formatted[formatted_key] = formatted_value
        else:
            # integer
            formatted[formatted_key] = value['uint']
    return formatted

# read user local state
def read_local_state(client, addr, app_id):
    results = client.account_info(addr)
    for local_state in results['apps-local-state']:
        if local_state['id'] == app_id:
            if 'key-value' not in local_state:
                return {}
            return format_state(local_state['key-value'])
    return {}

# read app global state
def read_global_state(client, addr, app_id):
    results = client.account_info(addr)
    apps_created = results['created-apps']
    for app in apps_created:
        if app['id'] == app_id:
            return format_state(app['params']['global-state'])
    return {}

# delete application
def delete_app(client, private_key, index):
    # declare sender
    sender = account.address_from_private_key(private_key)

	# get node suggested parameters
    params = client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 1000

    # create unsigned transaction
    txn = transaction.ApplicationDeleteTxn(sender, params, index)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # await confirmation
    wait_for_confirmation(client, tx_id)

    # display results
    transaction_response = client.pending_transaction_info(tx_id)
    print("Deleted app-id:", transaction_response['txn']['txn']['apid'])    

# close out from application
def close_out_app(client, private_key, index):
    # declare sender
    sender = account.address_from_private_key(private_key)

	# get node suggested parameters
    params = client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 1000

    # create unsigned transaction
    txn = transaction.ApplicationCloseOutTxn(sender, params, index)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # await confirmation
    wait_for_confirmation(client, tx_id)

    # display results
    transaction_response = client.pending_transaction_info(tx_id)
    print("Closed out from app-id: ",transaction_response['txn']['txn']['apid'])

# clear application
def clear_app(client, private_key, index):
    # declare sender
    sender = account.address_from_private_key(private_key)

	# get node suggested parameters
    params = client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 1000

    # create unsigned transaction
    txn = transaction.ApplicationClearStateTxn(sender, params, index)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # await confirmation
    wait_for_confirmation(client, tx_id)

    # display results
    transaction_response = client.pending_transaction_info(tx_id)
    print("Cleared app-id:", transaction_response['txn']['txn']['apid'])    

# convert 64 bit integer i to byte string
def intToBytes(i):
    return i.to_bytes(8, "big")

def main():
    # initialize an algodClient
#     algod_client = algod.AlgodClient(algod_token, algod_address)
    algod_client = algod.AlgodClient(algod_token, algod_address, headers=purestake_token)

    # define private keys
    creator_private_key = get_private_key_from_mnemonic(creator_mnemonic)
    user_private_key = get_private_key_from_mnemonic(user_mnemonic)

    # declare application state storage (immutable)
    local_ints = 0
    local_bytes = 1
    global_ints = 24 # 4 for setup + 20 for choices. Use a larger number for more choices.
    global_bytes = 1
    global_schema = transaction.StateSchema(global_ints, global_bytes)
    local_schema = transaction.StateSchema(local_ints, local_bytes)

    # get PyTeal approval program
    approval_program_ast = approval_program()
    # compile program to TEAL assembly
    approval_program_teal = compileTeal(approval_program_ast, mode=Mode.Application, version=3)
    # compile program to binary
    approval_program_compiled = compile_program(algod_client, approval_program_teal)

    # get PyTeal clear state program
    clear_state_program_ast = clear_state_program()
    # compile program to TEAL assembly
    clear_state_program_teal = compileTeal(clear_state_program_ast, mode=Mode.Application, version=3)
    # compile program to binary
    clear_state_program_compiled = compile_program(algod_client, clear_state_program_teal)

    # configure registration and voting period
    status = algod_client.status()
    regBegin = status['last-round'] + 10
    regEnd = regBegin + 10
    voteBegin = regEnd + 1
    voteEnd = voteBegin + 10

    print(f"Registration rounds: {regBegin} to {regEnd}")
    print(f"Vote rounds: {voteBegin} to {voteEnd}")

    # create list of bytes for app args
    app_args = [
        intToBytes(regBegin),
        intToBytes(regEnd),
        intToBytes(voteBegin),
        intToBytes(voteEnd)
    ]

    # create new application
    app_id = create_app(algod_client, creator_private_key, approval_program_compiled, clear_state_program_compiled, global_schema, local_schema, app_args)

    # read global state of application
    print("Global state:", read_global_state(algod_client, account.address_from_private_key(creator_private_key), app_id))

    # wait for registration period to start
    wait_for_round(algod_client, regBegin)

    # opt-in to application
    opt_in_app(algod_client, user_private_key, app_id)

    wait_for_round(algod_client, voteBegin)

    # call application without arguments
    call_app(algod_client, user_private_key, app_id, [b'vote', b'choiceA'])

    # read local state of application from user account
    print("Local state:", read_local_state(algod_client, account.address_from_private_key(user_private_key), app_id))

    # wait for registration period to start
    wait_for_round(algod_client, voteEnd)

    # read global state of application
    global_state = read_global_state(algod_client, account.address_from_private_key(creator_private_key), app_id)
    print("Global state:", global_state)

    max_votes = 0
    max_votes_choice = None
    for key,value in global_state.items():
        if key not in ('RegBegin', 'RegEnd', 'VoteBegin', 'VoteEnd', 'Creator') and isinstance(value, int):
            if value > max_votes:
                max_votes = value
                max_votes_choice = key
    
    print("The winner is:", max_votes_choice)

    # delete application
    delete_app(algod_client, creator_private_key, app_id)

    # clear application from user account
    clear_app(algod_client, user_private_key, app_id)

if __name__ == "__main__":
    main()

Registration rounds: 15762956 to 15762966
Vote rounds: 15762967 to 15762977
Waiting for confirmation...
Transaction DE4RMCGZ75S3GM3QIMJGRX6ZA7CYLOFF3N35ZW2Y7YAOHZPZ6E4A confirmed in round 15762949.
Created new app-id: 21288246
Global state: {'Creator': 'rXb6EkW955SFiwuw3nbC76PApIuuZFWAeB4gokMfI80=', 'VoteEnd': 15762977, 'RegBegin': 15762956, 'RegEnd': 15762966, 'VoteBegin': 15762967}
Waiting for round 15762956
Round 15762950
Round 15762951
Round 15762952
Round 15762953
Round 15762954
Round 15762955
Round 15762956
OptIn from account:  FFVTKNLA4J47EHONVR3U247LFLHE3TNATDEJLDRI2E73FF52BGVK7MFWHY
Waiting for confirmation...
Transaction ZKWHAHEWLAMMKWEB5PKJZFFMSXZP6SHUUEIW2GLSVMFLWZX6HWSA confirmed in round 15762959.
OptIn to app-id: 21288246
Waiting for round 15762967
Round 15762960
Round 15762961
Round 15762962
Round 15762963
Round 15762964
Round 15762965
Round 15762966
Round 15762967
Call from account: FFVTKNLA4J47EHONVR3U247LFLHE3TNATDEJLDRI2E73FF52BGVK7MFWHY
Waiting for confirmation...
